## Aggregate all necessary output files into reference datasets

### 0. Import necessary packages

In [2]:
import matsim
import os 
import pandas as pd
#import plotly.express as px 
# AttributeError: module 'numpy' has no attribute 'bool8'
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from collections import defaultdict
import plotly
import kaleido
%matplotlib inline
import plotly.io as pio


## 1. Runtime


####  Import run_info.csv for all sample sizes, alpha and stuck-time values and create data frames with relevent specifications:

#### 1 pct


In [3]:
run_time_1pct = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1pct-" + str(elem) + "-fCf_sCF_0.01_gS_default_3765/analysis/general/run_info.csv"
    temp = pd.read_csv(path, sep=",")
    start= pd.Timestamp(temp.iloc[6,1])
    end= pd.Timestamp(temp.iloc[7,1])
    diff = end - start
    run_time_1pct.append(diff)
run_time_1pct = pd.DataFrame({'runtime': run_time_1pct})
run_time_1pct.insert(1, 'sample_size', "1-pct")
run_time_1pct.insert(2, 'alpha', 1.0)
run_time_1pct.insert(3, 'stuck_time', 30.0)
run_time_1pct.insert(4,'global_seed', "default") 

In [4]:
run_time_1pct_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1pct-" + str(elem) + "-fCf_0.01_sCF_0.03162_gS_default_3765/analysis/general/run_info.csv"
    temp = pd.read_csv(path, sep=",")
    start= pd.Timestamp(temp.iloc[6,1])
    end= pd.Timestamp(temp.iloc[7,1])
    diff = end - start
    run_time_1pct_sCf.append(diff)
run_time_1pct_sCf = pd.DataFrame({'runtime': run_time_1pct_sCf})
run_time_1pct_sCf.insert(1, 'sample_size', "1-pct")
run_time_1pct_sCf.insert(2, 'alpha', 0.75)
run_time_1pct_sCf.insert(3, 'stuck_time', 30.0)
run_time_1pct_sCf.insert(4,'global_seed', "default") 

In [5]:
run_time_1pct_rGs = pd.DataFrame()

rGs = [ 4711,3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]
for seed in rGs:
    if (seed == 4711):
        global_seed = "rnd_" + str(seed)
        temp = {'runtime': run_time_1pct["runtime"].iloc[0], 'sample_size': '1-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': global_seed  }
        temp = pd.DataFrame(data=temp, index=[rGs.index(4711)])
        run_time_1pct_rGs = pd.concat([run_time_1pct_rGs, temp])
    elif (seed == 3254):
        path ="/home/lola/math_cluster/output/output-lausitz-1pct-1-fCf_sCF_0.01_gS_3254_3765/analysis/general/run_info.csv"
        temp = pd.read_csv(path, sep = ",")
        start= pd.Timestamp(temp.iloc[6,1])
        end= pd.Timestamp(temp.iloc[7,1])
        diff = end - start
        temp2 = pd.DataFrame({'runtime': diff, 'sample_size': '1-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': "rnd_" + str(seed)  }, index = [rGs.index(3254)])
        run_time_1pct_rGs = pd.concat([run_time_1pct_rGs, temp2], axis = 0)
    else:
        path = "/home/lola/math_cluster/output/output-lausitz-1.0-pct-1-fCf_sCF_0.01_gS_"+str(seed) + "_3765/analysis/general/run_info.csv"
        temp = pd.read_csv(path, sep = ",")
        start= pd.Timestamp(temp.iloc[6,1])
        end= pd.Timestamp(temp.iloc[7,1])
        diff = end - start
        temp2 = pd.DataFrame({'runtime': diff, 'sample_size': '1-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': "rnd_" + str(seed)  }, index = [rGs.index(seed)] )
        run_time_1pct_rGs = pd.concat([run_time_1pct_rGs, temp2], axis = 0)

In [6]:
run_time_1pct_sT = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-"+str(elem)+"-fCf_sCF_0.01_gS_4711_sT_3000.0_3765/analysis/general/run_info.csv"
    temp = pd.read_csv(path, sep = ",")
    start= pd.Timestamp(temp.iloc[6,1])
    end= pd.Timestamp(temp.iloc[7,1])
    diff = end - start
    run_time_1pct_sT.append(diff)
run_time_1pct_sT = pd.DataFrame({'runtime': run_time_1pct_sT})
run_time_1pct_sT.insert(1, 'sample_size', "1-pct")
run_time_1pct_sT.insert(2, 'alpha', 1.0)
run_time_1pct_sT.insert(3, 'stuck_time', 3000.0)
run_time_1pct_sT.insert(4,'global_seed', "default") 

In [7]:
run_time_1pct_sT_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-"+str(elem)+"-fCf_0.01_sCF_0.03162_gS_4711_sT_3000.0_3765/analysis/general/run_info.csv"
    temp = pd.read_csv(path, sep = ",")
    start= pd.Timestamp(temp.iloc[6,1])
    end= pd.Timestamp(temp.iloc[7,1])
    diff = end - start
    run_time_1pct_sT_sCf.append(diff)
run_time_1pct_sT_sCf = pd.DataFrame({'runtime': run_time_1pct_sT_sCf})
run_time_1pct_sT_sCf.insert(1, 'sample_size', "1-pct")
run_time_1pct_sT_sCf.insert(2, 'alpha', 0.75)
run_time_1pct_sT_sCf.insert(3, 'stuck_time', 3000.0)
run_time_1pct_sT_sCf.insert(4,'global_seed', "default")    
    

#### 5pct

In [8]:
run_time_5pct = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(elem) + "-fCf_sCF_0.05_gS_4711_3765/analysis/general/run_info.csv"
    temp = pd.read_csv(path, sep=",")
    start= pd.Timestamp(temp.iloc[6,1])
    end= pd.Timestamp(temp.iloc[7,1])
    diff = end - start
    run_time_5pct.append(diff)
run_time_5pct = pd.DataFrame({'runtime': run_time_5pct})
run_time_5pct.insert(1, 'sample_size', "5-pct")
run_time_5pct.insert(2, 'alpha', 1.0)
run_time_5pct.insert(3, 'stuck_time', 30.0)
run_time_5pct.insert(4,'global_seed', "default") 

In [9]:
run_time_5pct_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(elem) + "-fCf_0.05_sCF_0.10574_gS_4711_3765/analysis/general/run_info.csv"
    temp = pd.read_csv(path, sep=",")
    start= pd.Timestamp(temp.iloc[6,1])
    end= pd.Timestamp(temp.iloc[7,1])
    diff = end - start
    run_time_5pct_sCf.append(diff)
run_time_5pct_sCf = pd.DataFrame({'runtime': run_time_5pct_sCf})
run_time_5pct_sCf.insert(1, 'sample_size', "5-pct")
run_time_5pct_sCf.insert(2, 'alpha', 0.75)
run_time_5pct_sCf.insert(3, 'stuck_time', 30.0)
run_time_5pct_sCf.insert(4,'global_seed', "default")

In [10]:
run_time_5pct_rGs = pd.DataFrame()
rGs = [4711, 3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]

for seed in rGs:
    if (seed ==4711):
        global_seed = "rnd_" + str(seed)
        temp = {'runtime': run_time_5pct["runtime"].iloc[0], 'sample_size': '5-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': global_seed  }
        temp = pd.DataFrame(data=temp, index=[rGs.index(4711)])
        run_time_5pct_rGs = pd.concat([run_time_5pct_rGs, temp])
    else:
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-1-fCf_sCF_0.05_gS_"+ str(seed) +"_3765/analysis/general/run_info.csv"
        temp = pd.read_csv(path, sep = ",")
        start= pd.Timestamp(temp.iloc[6,1])
        end= pd.Timestamp(temp.iloc[7,1])
        diff = end - start
        temp2 = pd.DataFrame({'runtime': diff, 'sample_size': '5-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': "rnd_" + str(seed)  }, index = [rGs.index(seed)])
        run_time_5pct_rGs = pd.concat([run_time_5pct_rGs, temp2], axis = 0)

In [11]:
run_time_5pct_sT = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-"+ str(elem) +"-fCf_sCF_0.05_gS_4711_sT_600.0_3765/analysis/general/run_info.csv"
    temp = pd.read_csv(path, sep = ",")
    start= pd.Timestamp(temp.iloc[6,1])
    end= pd.Timestamp(temp.iloc[7,1])
    diff = end - start
    run_time_5pct_sT.append(diff)
run_time_5pct_sT = pd.DataFrame({'runtime': run_time_5pct_sT})
run_time_5pct_sT.insert(1, 'sample_size', "5-pct")
run_time_5pct_sT.insert(2, 'alpha', 1.0)
run_time_5pct_sT.insert(3, 'stuck_time', 600.0)
run_time_5pct_sT.insert(4,'global_seed', "default")

In [12]:
run_time_5pct_sT_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-"+str(elem)+"-fCf_0.05_sCF_0.10574_gS_4711_sT_600.0_3765/analysis/general/run_info.csv"
    temp = pd.read_csv(path,sep = ",")
    start= pd.Timestamp(temp.iloc[6,1])
    end= pd.Timestamp(temp.iloc[7,1])
    diff = end - start
    run_time_5pct_sT_sCf.append(diff)
run_time_5pct_sT_sCf = pd.DataFrame({'runtime': run_time_5pct_sT_sCf})
run_time_5pct_sT_sCf.insert(1, 'sample_size', "5-pct")
run_time_5pct_sT_sCf.insert(2, 'alpha', 0.75)
run_time_5pct_sT_sCf.insert(3, 'stuck_time', 600.0)
run_time_5pct_sT_sCf.insert(4,'global_seed', "default")
    

#### 10 pct #### 


In [13]:
run_time_10pct = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" + str(elem) + "-fCf_sCF_0.1_gS_4711_3765/analysis/general/run_info.csv"
    temp = pd.read_csv(path, sep=",")
    start= pd.Timestamp(temp.iloc[6,1])
    end= pd.Timestamp(temp.iloc[7,1])
    diff = end - start
    run_time_10pct.append(diff)
run_time_10pct = pd.DataFrame({'runtime': run_time_10pct})
run_time_10pct.insert(1, 'sample_size', "10-pct")
run_time_10pct.insert(2, 'alpha', 1.0)
run_time_10pct.insert(3, 'stuck_time', 30.0)
run_time_10pct.insert(4,'global_seed', "default") 

In [14]:
run_time_10pct_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" + str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_3765/analysis/general/run_info.csv"
    temp = pd.read_csv(path, sep=",")
    start= pd.Timestamp(temp.iloc[6,1])
    end= pd.Timestamp(temp.iloc[7,1])
    diff = end - start
    run_time_10pct_sCf.append(diff)
run_time_10pct_sCf = pd.DataFrame({'runtime': run_time_10pct_sCf})
run_time_10pct_sCf.insert(1, 'sample_size', "10-pct")
run_time_10pct_sCf.insert(2, 'alpha', 0.75)
run_time_10pct_sCf.insert(3, 'stuck_time', 30.0)
run_time_10pct_sCf.insert(4,'global_seed', "default") 

In [15]:

run_time_10pct_sT = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-"+ str(elem) + "-fCf_sCF_0.1_gS_4711_sT_300.0_3765/analysis/general/run_info.csv"
    temp = pd.read_csv(path, sep=",")
    start= pd.Timestamp(temp.iloc[6,1])
    end= pd.Timestamp(temp.iloc[7,1])
    diff = end - start
    run_time_10pct_sT.append(diff)
run_time_10pct_sT = pd.DataFrame({'runtime': run_time_10pct_sT})
run_time_10pct_sT.insert(1, 'sample_size', "10-pct")
run_time_10pct_sT.insert(2, 'alpha', 1.0)
run_time_10pct_sT.insert(3, 'stuck_time', 300.0)
run_time_10pct_sT.insert(4,'global_seed', "default") 

In [16]:
run_time_10pct_sT_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-" +str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_sT_300.0_3765/analysis/general/run_info.csv"
    temp = pd.read_csv(path, sep=",")
    start= pd.Timestamp(temp.iloc[6,1])
    end= pd.Timestamp(temp.iloc[7,1])
    diff = end - start
    run_time_10pct_sT_sCf.append(diff)
run_time_10pct_sT_sCf = pd.DataFrame({'runtime': run_time_10pct_sT_sCf})
run_time_10pct_sT_sCf.insert(1, 'sample_size', "10-pct")
run_time_10pct_sT_sCf.insert(2, 'alpha', 0.75)
run_time_10pct_sT_sCf.insert(3, 'stuck_time', 300.0)
run_time_10pct_sT_sCf.insert(4,'global_seed', "default") 


#### 25 pct

In [17]:
path =  "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765/analysis/general/run_info.csv"
temp = pd.read_csv(path, sep = ",")
start= pd.Timestamp(temp.iloc[6,1])
end= pd.Timestamp(temp.iloc[7,1])
diff = end - start
run_time_25pct = pd.DataFrame({'runtime': diff, 'sample_size': '25-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': "default"  }, index = [0])

In [18]:
path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765/analysis/general/run_info.csv"
temp = pd.read_csv(path, sep = ",")
start= pd.Timestamp(temp.iloc[6,1])
end= pd.Timestamp(temp.iloc[7,1])
diff = end - start
run_time_25pct_sCf = pd.DataFrame({'runtime': diff, 'sample_size': '25-pct', 'alpha': 0.75, 'stuck_time': 30.0, 'global_seed': "default"  }, index = [1])

In [19]:
path = "/home/lola/math_cluster/output/output-lausitz-25-pct-1-fCf_sCF_0.25_gS_4711_sT_120.0_3765/analysis/general/run_info.csv"
temp = pd.read_csv(path, sep = ",")
start= pd.Timestamp(temp.iloc[6,1])
end= pd.Timestamp(temp.iloc[7,1])
diff = end - start
run_time_25pct_sT = pd.DataFrame({'runtime': diff, 'sample_size': '25-pct', 'alpha': 1.0, 'stuck_time': 120.0, 'global_seed': "default"  }, index = [2])

In [20]:
path = "/home/lola/math_cluster/output/output-lausitz-25-pct-1-fCf_0.25_sCF_0.35355_gS_4711_sT_120.0_3765/analysis/general/run_info.csv"
temp = pd.read_csv(path, sep = ",")
start= pd.Timestamp(temp.iloc[6,1])
end= pd.Timestamp(temp.iloc[7,1])
diff = end - start
run_time_25pct_sT_sCf = pd.DataFrame({'runtime': diff, 'sample_size': '25-pct', 'alpha': 0.75, 'stuck_time': 120.0, 'global_seed': "default"  }, index = [3])

#### 50 pct

In [21]:
path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765/analysis/general/run_info.csv"
temp = pd.read_csv(path, sep = ",")
start= pd.Timestamp(temp.iloc[6,1])
end= pd.Timestamp(temp.iloc[7,1])
diff = end - start
run_time_50pct = pd.DataFrame({'runtime': diff, 'sample_size': '50-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': "default"  }, index = [0])

In [22]:
path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_0.5_sCF_0.5946_gS_4711_3765/analysis/general/run_info.csv"
temp = pd.read_csv(path, sep = ",")
start= pd.Timestamp(temp.iloc[6,1])
end= pd.Timestamp(temp.iloc[7,1])
diff = end - start
run_time_50pct_sCf = pd.DataFrame({'runtime': diff, 'sample_size': '50-pct', 'alpha': 0.75, 'stuck_time': 30.0, 'global_seed': "default"  }, index = [1])

In [23]:
path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_sCF_0.5_gS_4711_sT_60.0_3765/analysis/general/run_info.csv"
temp = pd.read_csv(path, sep = ",")
start= pd.Timestamp(temp.iloc[6,1])
end= pd.Timestamp(temp.iloc[7,1])
diff = end - start
run_time_50pct_sT = pd.DataFrame({'runtime': diff, 'sample_size': '50-pct', 'alpha': 1.0, 'stuck_time': 60.0, 'global_seed': "default"  }, index = [2])

In [24]:
path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_0.5_sCF_0.5946_gS_4711_sT_60.0_3765/analysis/general/run_info.csv"
temp = pd.read_csv(path, sep = ",")
start= pd.Timestamp(temp.iloc[6,1])
end= pd.Timestamp(temp.iloc[7,1])
diff = end - start
run_time_50pct_sT_sCf = pd.DataFrame({'runtime': diff, 'sample_size': '50-pct', 'alpha': 0.75, 'stuck_time': 60.0, 'global_seed': "default"  }, index = [3])

In [25]:
path = "/home/lola/Nextcloud/Masterarbeit/03_Outputs_From_RunsLausitz/output-lausitz-25-pct-doubled-fCf_0.5_sCF_0.5_gS_4711_3765/analysis/general/run_info.csv"
temp = pd.read_csv(path, sep = ",")
start= pd.Timestamp(temp.iloc[6,1])
end= pd.Timestamp(temp.iloc[7,1])
diff = end - start
run_time_25_pct_doubled = pd.DataFrame({'runtime': diff, 'sample_size': '25-pct-doubled', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': "default"  }, index = [4])

#### 100 pct

In [26]:
path =  "/home/lola/math_cluster/output/output-lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765/analysis/general/run_info.csv"
temp = pd.read_csv(path, sep = ",")
start= pd.Timestamp(temp.iloc[6,1])
end= pd.Timestamp(temp.iloc[7,1])
diff = end - start
run_time_100pct = pd.DataFrame({'runtime': diff, 'sample_size': '100-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': "default"  }, index = [0])


In [27]:
path = "/home/lola/Nextcloud/Masterarbeit/03_Outputs_From_RunsLausitz/output-lausitz-25.0-pct-quadrupled-fCf_1.0_sCF_1.0_gS_4711_3765/analysis/general/run_info.csv"
temp = pd.read_csv(path, sep = ",")
start= pd.Timestamp(temp.iloc[6,1])
end= pd.Timestamp(temp.iloc[7,1])
diff = end - start
run_time_25_pct_quadrupled = pd.DataFrame({'runtime': diff, 'sample_size': '25-pct-quadrupled', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': "default"  }, index = [4])

#### pd.Concat,  convert minutes into decimal base?! and write csv

In [28]:
# concat !
runtimes_1_100 = pd.concat([run_time_1pct, run_time_1pct_sCf, run_time_1pct_sT, run_time_1pct_sT_sCf, run_time_1pct_rGs,
                            run_time_5pct,run_time_5pct_sCf, run_time_5pct_sT, run_time_5pct_sT_sCf, run_time_5pct_rGs,
                            run_time_10pct,run_time_10pct_sCf, run_time_10pct_sT, run_time_10pct_sT_sCf,                  
                              run_time_25pct,run_time_25pct_sCf, run_time_25pct_sT, run_time_25pct_sT_sCf,
                                run_time_50pct,run_time_50pct_sCf, run_time_50pct_sT, run_time_50pct_sT_sCf,run_time_25_pct_doubled,
                                run_time_100pct, run_time_25_pct_quadrupled  ])






In [29]:
# transform minutes into decimal 
runtimes_1_100['runtime'] =  runtimes_1_100.runtime.astype('timedelta64[s]')/  pd.Timedelta(minutes=60)

In [30]:
runtimes_1_100 = runtimes_1_100.iloc[:, [1, 2, 3, 4,0]]
runtimes_1_100.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/runtimes_1_to_100pct_correct.csv', index = False) 

## 2. Number of stuck-time violations

#### Function to filter the output events and count number of stuck-time violatins per vehicle

In [ ]:
# Number of stuck vehicles
def countStuckVehicles(pathToEvents):
    events_file = pathToEvents
    #"/home/lola/math_cluster/output/output-lausitz-1pct-1-fCf_sCF_0.01_gS_default_3765/lausitz-1pct-1-fCf_sCf_0.01_gS_default_3765.output_events.xml.gz"

    # Read events - filter and return the listed event types only
    events = matsim.event_reader(
        events_file,
        types="stuckAndContinue",
    )


    stuck_person = defaultdict(list)

    # Loop on all filtered events
    for event in events:
        if event["type"] == "stuckAndContinue":
            stuck_person[event["person"]].append(event["legMode"])
    
    count = 0 
    for key in stuck_person.keys():
        count+= len(stuck_person[key])
    
    return count


#### 1pct

In [ ]:
stuck_veh_1pct = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1pct-" + str(elem) + "-fCf_sCF_0.01_gS_default_3765/lausitz-1pct-"+str(elem)+ "-fCf_sCf_0.01_gS_default_3765.output_events.xml.gz"
    stuck_veh = countStuckVehicles(path)
    stuck_veh_1pct.append(stuck_veh)
stuck_veh_1pct = pd.DataFrame({'n_stuck_veh': stuck_veh_1pct})
stuck_veh_1pct.insert(1, 'sample_size', "1-pct")
stuck_veh_1pct.insert(2, 'alpha', 1.0)
stuck_veh_1pct.insert(3, 'global_seed', "default")
stuck_veh_1pct.insert(4, 'stuck_time', 30.0)

In [ ]:
stuck_veh_1pct_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1pct-" + str(elem) + "-fCf_0.01_sCF_0.03162_gS_default_3765/lausitz-1pct-" +str(elem)+ "-fCf_0.01_sCf_0.03162_gS_default_3765.output_events.xml.gz"
    stuck_veh = countStuckVehicles(path)
    stuck_veh_1pct_sCf.append(stuck_veh)
stuck_veh_1pct_sCf = pd.DataFrame({'n_stuck_veh': stuck_veh_1pct_sCf})
stuck_veh_1pct_sCf.insert(1, 'sample_size', "1-pct")
stuck_veh_1pct_sCf.insert(2, 'alpha', 0.75)
stuck_veh_1pct_sCf.insert(3, 'global_seed', "default")
stuck_veh_1pct_sCf.insert(4, 'stuck_time', 30.0)

In [ ]:
# stuck-time of it0

# path
path = "/home/lolat/math_cluster/output/output-lausitz-1pct-1-fCf_sCF_0.01_gS_default_3765/ITERS/it.0/lausitz-1pct-1-fCf_sCf_0.01_gS_default_3765.0.events.xml.gz"


st_veh_s1_it0 = countStuckVehicles(path)
# 3030

# number of stuck vehicles after iteration 500 : 1846

In [4]:
st_veh_s1_it0 

3030

In [45]:
#  1 pct random seed
stuck_veh_1pct_rGs = pd.DataFrame()
rGs = [ 4711,3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]
for seed in rGs:
    if (seed == 4711):
        global_seed = "rnd_" + str(seed)
        # insert number of stuck time violations from the first 1 pct sample
        temp = {'n_stuck_veh': stuck_veh_1pct["n_stuck_veh"].iloc[0], 'sample_size': '1-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': global_seed  }
        temp = pd.DataFrame(data=temp, index=[rGs.index(4711)])
        stuck_veh_1pct_rGs = pd.concat([stuck_veh_1pct_rGs, temp])
    elif (seed == 3254):
        path ="/home/lola/math_cluster/output/output-lausitz-1pct-1-fCf_sCF_0.01_gS_3254_3765/lausitz-1pct-1-fCf_sCf_0.01_gS_3254_3765.output_events.xml.gz"
        stuck_veh = countStuckVehicles(path)
        temp2 = pd.DataFrame({'n_stuck_veh': stuck_veh, 'sample_size': '1-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': "rnd_" + str(seed)  }, index = [rGs.index(3254)])
        stuck_veh_1pct_rGs = pd.concat([stuck_veh_1pct_rGs, temp2], axis = 0)
    else:
        path = "/home/lola/math_cluster/output/output-lausitz-1.0-pct-1-fCf_sCF_0.01_gS_" +str(seed) + "_3765/lausitz-1.0-pct-1-fCf_sCF_0.01_gS_" + str(seed) + "_3765.output_events.xml.gz"
        stuck_veh = countStuckVehicles(path)
        temp2 = pd.DataFrame({'n_stuck_veh': stuck_veh, 'sample_size': '1-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': "rnd_" + str(seed)  }, index = [rGs.index(seed)] )
        stuck_veh_1pct_rGs = pd.concat([stuck_veh_1pct_rGs, temp2], axis = 0)

In [46]:
# 1 pct, alpha = 1, sT scaled
stuck_veh_1pct_sT = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-" + str(elem) + "-fCf_sCF_0.01_gS_4711_sT_3000.0_3765/lausitz-1-pct-" + str(elem) + "-fCf_sCF_0.01_gS_4711_sT_3000.0_3765.output_events.xml.gz"
    stuck_veh = countStuckVehicles(path)
    stuck_veh_1pct_sT.append(stuck_veh)
stuck_veh_1pct_sT = pd.DataFrame({'n_stuck_veh': stuck_veh_1pct_sT})
stuck_veh_1pct_sT.insert(1, 'sample_size', "1-pct")
stuck_veh_1pct_sT.insert(2, 'alpha', 1.0)
stuck_veh_1pct_sT.insert(3, 'global_seed', "default")
stuck_veh_1pct_sT.insert(4, 'stuck_time', 3000.0)


In [47]:
# 1 pct, alpha = 0.75, sT scaled
stuck_veh_1pct_sT_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-" + str(elem) + "-fCf_0.01_sCF_0.03162_gS_4711_sT_3000.0_3765/lausitz-1-pct-" + str(elem) +"-fCf_0.01_sCF_0.03162_gS_4711_sT_3000.0_3765.output_events.xml.gz"
    stuck_veh = countStuckVehicles(path)
    stuck_veh_1pct_sT_sCf.append(stuck_veh)
stuck_veh_1pct_sT_sCf = pd.DataFrame({'n_stuck_veh': stuck_veh_1pct_sT_sCf})
stuck_veh_1pct_sT_sCf.insert(1, 'sample_size', "1-pct")
stuck_veh_1pct_sT_sCf.insert(2, 'alpha', 0.75)
stuck_veh_1pct_sT_sCf.insert(3, 'global_seed', "default")
stuck_veh_1pct_sT_sCf.insert(4, 'stuck_time', 3000.0)

In [48]:
stuck_time_violation_1pct = pd.concat([stuck_veh_1pct, stuck_veh_1pct_sCf, stuck_veh_1pct_sT, stuck_veh_1pct_sT_sCf, stuck_veh_1pct_rGs])

In [51]:
stuck_time_violation_1pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_1_pct.csv', index = False) 

#### 5 pct

In [23]:
# 5 pct, alpha = 1
stuck_veh_5pct = []
for elem in range(1,11,1):
    if (elem==6):
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765/lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765-2.output_events.xml.gz"
        stuck_veh = countStuckVehicles(path)
        stuck_veh_5pct.append(stuck_veh)
    else:
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(elem) + "-fCf_sCF_0.05_gS_4711_3765/lausitz-5.0-pct-"+str(elem)+ "-fCf_sCF_0.05_gS_4711_3765.output_events.xml.gz"
        stuck_veh = countStuckVehicles(path)
        stuck_veh_5pct.append(stuck_veh)
stuck_veh_5pct = pd.DataFrame({'n_stuck_veh': stuck_veh_5pct})

stuck_veh_5pct.insert(1, 'sample_size', "5-pct")
stuck_veh_5pct.insert(2, 'alpha', 1.0)
stuck_veh_5pct.insert(3, 'global_seed', "default")
stuck_veh_5pct.insert(4, 'stuck_time', 30.0)

In [24]:
# 5 pct, alpha = 0.75
stuck_veh_5pct_sCf = []
for elem in range(1,11,1):
    if (elem==6):
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-6-fCf_0.05_sCF_0.10574_gS_4711_3765/lausitz-5.0-pct-6-fCf_0.05_sCF_0.10574_gS_4711_3765-2.output_events.xml.gz"
        stuck_veh = countStuckVehicles(path)
        stuck_veh_5pct_sCf.append(stuck_veh)
    else:
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(elem) + "-fCf_0.05_sCF_0.10574_gS_4711_3765/lausitz-5.0-pct-"+str(elem)+ "-fCf_0.05_sCF_0.10574_gS_4711_3765.output_events.xml.gz"
        stuck_veh = countStuckVehicles(path)
        stuck_veh_5pct_sCf.append(stuck_veh)
stuck_veh_5pct_sCf = pd.DataFrame({'n_stuck_veh': stuck_veh_5pct_sCf})

stuck_veh_5pct_sCf.insert(1, 'sample_size', "5-pct")
stuck_veh_5pct_sCf.insert(2, 'alpha', 0.75)
stuck_veh_5pct_sCf.insert(3, 'global_seed', "default")
stuck_veh_5pct_sCf.insert(4, 'stuck_time', 30.0)


In [25]:
# 5 pct random global seed, alpha = 1
stuck_veh_5pct_rGs = pd.DataFrame()
rGs = [4711, 3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]

for seed in rGs:
    if (seed ==4711):
        global_seed = "rnd_" + str(seed)
        temp = {'n_stuck_veh': stuck_veh_5pct["n_stuck_veh"].iloc[0], 'sample_size': '5-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': global_seed  }
        temp = pd.DataFrame(data=temp, index=[rGs.index(4711)])
        stuck_veh_5pct_rGs = pd.concat([stuck_veh_5pct_rGs, temp])
    else:
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-1-fCf_sCF_0.05_gS_" + str(seed) + "_3765/lausitz-5.0-pct-1-fCf_sCF_0.05_gS_"+ str(seed) +"_3765.output_events.xml.gz"
        stuck_veh = countStuckVehicles(path)
        temp2 = pd.DataFrame({'n_stuck_veh': stuck_veh, 'sample_size': '5-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': "rnd_" + str(seed)  }, index = [rGs.index(seed)])
        stuck_veh_5pct_rGs = pd.concat([stuck_veh_5pct_rGs, temp2], axis = 0)


In [26]:
# 5 pct, alpha = 1.0, sT scaled
stuck_veh_5pct_sT = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-" + str(elem) + "-fCf_sCF_0.05_gS_4711_sT_600.0_3765/lausitz-5-pct-" + str(elem) +"-fCf_sCF_0.05_gS_4711_sT_600.0_3765.output_events.xml.gz"
    stuck_veh = countStuckVehicles(path)
    stuck_veh_5pct_sT.append(stuck_veh)
stuck_veh_5pct_sT = pd.DataFrame({'n_stuck_veh': stuck_veh_5pct_sT})

stuck_veh_5pct_sT.insert(1, 'sample_size', "5-pct")
stuck_veh_5pct_sT.insert(2, 'alpha', 1.0)
stuck_veh_5pct_sT.insert(3, 'global_seed', "default")
stuck_veh_5pct_sT.insert(4, 'stuck_time', 600.0)

In [27]:
# 5 pct, alpha = 0.75, sT scaled
stuck_veh_5pct_sT_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-" + str (elem) + "-fCf_0.05_sCF_0.10574_gS_4711_sT_600.0_3765/lausitz-5-pct-" + str(elem) + "-fCf_0.05_sCF_0.10574_gS_4711_sT_600.0_3765.output_events.xml.gz"
    stuck_veh = countStuckVehicles(path)
    stuck_veh_5pct_sT_sCf.append(stuck_veh)
stuck_veh_5pct_sT_sCf = pd.DataFrame({'n_stuck_veh': stuck_veh_5pct_sT_sCf})

stuck_veh_5pct_sT_sCf.insert(1, 'sample_size', "5-pct")
stuck_veh_5pct_sT_sCf.insert(2, 'alpha', 0.75)
stuck_veh_5pct_sT_sCf.insert(3, 'global_seed', "default")
stuck_veh_5pct_sT_sCf.insert(4, 'stuck_time', 600.0)

In [35]:
stuck_time_violation_5pct = pd.concat([stuck_veh_5pct, stuck_veh_5pct_sCf, stuck_veh_5pct_sT, stuck_veh_5pct_sT_sCf, stuck_veh_5pct_rGs])

In [36]:
stuck_time_violation_5pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_5_pct.csv', index = False) 

#### 10 pct

In [29]:
# 10 pct, alpha = 1
stuck_veh_10pct = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" + str(elem) + "-fCf_sCF_0.1_gS_4711_3765/lausitz-10.0-pct-" + str(elem) + "-fCf_sCF_0.1_gS_4711_3765.output_events.xml.gz"
    stuck_veh = countStuckVehicles(path)
    stuck_veh_10pct.append(stuck_veh)
stuck_veh_10pct = pd.DataFrame({'n_stuck_veh': stuck_veh_10pct})
stuck_veh_10pct.insert(1, 'sample_size', "10-pct")
stuck_veh_10pct.insert(2, 'alpha', 1.0)
stuck_veh_10pct.insert(3, 'global_seed', "default")
stuck_veh_10pct.insert(4, 'stuck_time', 30.0)


In [31]:
# 10 pct, alpha = 0.75
stuck_veh_10pct_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-"+ str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_3765/lausitz-10.0-pct-" + str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_3765.output_events.xml.gz"
    stuck_veh = countStuckVehicles(path)
    stuck_veh_10pct_sCf.append(stuck_veh)
stuck_veh_10pct_sCf = pd.DataFrame({'n_stuck_veh': stuck_veh_10pct_sCf})
stuck_veh_10pct_sCf.insert(1, 'sample_size', "10-pct")
stuck_veh_10pct_sCf.insert(2, 'alpha', 0.75)
stuck_veh_10pct_sCf.insert(3, 'global_seed', "default")
stuck_veh_10pct_sCf.insert(4, 'stuck_time', 30.0)

In [ ]:
# 10pct, alpha = 1 , sT scaled
stuck_veh_10pct_sT = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-" + str(elem) + "-fCf_sCF_0.1_gS_4711_sT_300.0_3765/lausitz-10-pct-" + str(elem) + "-fCf_sCF_0.1_gS_4711_sT_300.0_3765.output_events.xml.gz"
    stuck_veh = countStuckVehicles(path)
    stuck_veh_10pct_sT.append(stuck_veh)
stuck_veh_10pct_sT = pd.DataFrame({'n_stuck_veh': stuck_veh_10pct_sT})
stuck_veh_10pct_sT.insert(1, 'sample_size', "10-pct")
stuck_veh_10pct_sT.insert(2, 'alpha', 1.0)
stuck_veh_10pct_sT.insert(3, 'global_seed', "default")
stuck_veh_10pct_sT.insert(4, 'stuck_time', 300.0)

In [ ]:
# 10 pct, alpha = 0.75, sT scaled
# 10pct, alpha = 1 , sT scaled
stuck_veh_10pct_sT_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-" + str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_sT_300.0_3765/lausitz-10-pct-" + str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_sT_300.0_3765.output_events.xml.gz"
    stuck_veh = countStuckVehicles(path)
    stuck_veh_10pct_sT_sCf.append(stuck_veh)
stuck_veh_10pct_sT_sCf = pd.DataFrame({'n_stuck_veh': stuck_veh_10pct_sT_sCf})
stuck_veh_10pct_sT_sCf.insert(1, 'sample_size', "10-pct")
stuck_veh_10pct_sT_sCf.insert(2, 'alpha', 0.75)
stuck_veh_10pct_sT_sCf.insert(3, 'global_seed', "default")
stuck_veh_10pct_sT_sCf.insert(4, 'stuck_time', 300.0)

In [38]:
stuck_time_violation_10pct = pd.concat([stuck_veh_10pct, stuck_veh_10pct_sCf, stuck_veh_10pct_sT, stuck_veh_10pct_sT_sCf])

In [39]:
stuck_time_violation_10pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_10_pct.csv', index = False) 

#### 25 pct

In [ ]:
# 25 pct, alpha = 1
path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765/lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765.output_events.xml.gz"
stuck_veh = countStuckVehicles(path)
stuck_veh_25pct = pd.DataFrame({'n_stuck_veh': stuck_veh,'sample_size': "25-pct", 'alpha': 1.0,
                                'global_seed': "default", 'stuck_time': 30.0 }, index= [0])




In [ ]:
# 25 pct, alpha = 0.75
path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765/lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765.output_events.xml.gz"
stuck_veh = countStuckVehicles(path)
stuck_veh_25pct_sCf = pd.DataFrame({'n_stuck_veh': stuck_veh,'sample_size': "25-pct", 'alpha': 0.75,
                                'global_seed': "default", 'stuck_time': 30.0 }, index = [1])


In [44]:
# 25 pct, alpha = 1, sT scaled
path = "/home/lola/math_cluster/output/output-lausitz-25-pct-1-fCf_sCF_0.25_gS_4711_sT_120.0_3765/lausitz-25-pct-1-fCf_sCF_0.25_gS_4711_sT_120.0_3765.output_events.xml.gz"
stuck_veh = countStuckVehicles(path)
stuck_veh_25pct_sT = pd.DataFrame({'n_stuck_veh': stuck_veh,'sample_size': "25-pct", 'alpha': 1.0,
                                'global_seed': "default", 'stuck_time': 120.0 }, index = [2])

In [45]:
# 25 pct, alpha = 0.75, sT scaled
path = "/home/lola/math_cluster/output/output-lausitz-25-pct-1-fCf_0.25_sCF_0.35355_gS_4711_sT_120.0_3765/lausitz-25-pct-1-fCf_0.25_sCF_0.35355_gS_4711_sT_120.0_3765.output_events.xml.gz"
stuck_veh = countStuckVehicles(path)
stuck_veh_25pct_sT_sCf = pd.DataFrame({'n_stuck_veh': stuck_veh,'sample_size': "25-pct", 'alpha': 0.75,
                                'global_seed': "default", 'stuck_time': 120.0 }, index = [3])

In [46]:
stuck_time_violation_25pct = pd.concat([stuck_veh_25pct, stuck_veh_25pct_sCf, stuck_veh_25pct_sT, stuck_veh_25pct_sT_sCf])

In [47]:
stuck_time_violation_25pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_25_pct.csv', index = False) 

#### 50 pct

In [ ]:
# 50 pct, alpha = 1
path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765/lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765.output_events.xml.gz"
stuck_veh = countStuckVehicles(path)
stuck_veh_50pct = pd.DataFrame({'n_stuck_veh': stuck_veh,'sample_size': "50-pct", 'alpha': 1.0,
                                'global_seed': "default", 'stuck_time': 30.0 }, index = [0])


In [51]:
# 50 pct, alpha = 0.75
path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_0.5_sCF_0.5946_gS_4711_3765/lausitz-50.0-pct-fCf_0.5_sCF_0.5946_gS_4711_3765.output_events.xml.gz"
stuck_veh = countStuckVehicles(path)
stuck_veh_50pct_sCf = pd.DataFrame({'n_stuck_veh': stuck_veh,'sample_size': "50-pct", 'alpha': 0.75,
                                'global_seed': "default", 'stuck_time': 30.0 }, index = [1])

In [52]:
# 50 pct, alpha = 1, sT scaled
path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_sCF_0.5_gS_4711_sT_60.0_3765/lausitz-50-pct-1-fCf_sCF_0.5_gS_4711_sT_60.0_3765.output_events.xml.gz"
stuck_veh = countStuckVehicles(path)
stuck_veh_50pct_sT = pd.DataFrame({'n_stuck_veh': stuck_veh,'sample_size': "50-pct", 'alpha': 1.0,
                                'global_seed': "default", 'stuck_time': 60.0 }, index = [2])

In [53]:
# 50 pct, alpha = 0.75, sT scaled
path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_0.5_sCF_0.5946_gS_4711_sT_60.0_3765/lausitz-50-pct-1-fCf_0.5_sCF_0.5946_gS_4711_sT_60.0_3765.output_events.xml.gz"
stuck_veh = countStuckVehicles(path)
stuck_veh_50pct_sT_sCf = pd.DataFrame({'n_stuck_veh': stuck_veh,'sample_size': "50-pct", 'alpha': 0.75,
                                'global_seed': "default", 'stuck_time': 60.0 }, index = [3])

In [ ]:
stuck_time_violation_50pct = pd.concat([stuck_veh_50pct, stuck_veh_50pct_sCf, stuck_veh_50pct_sT, stuck_veh_50pct_sT_sCf])
stuck_time_violation_50pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_50_pct.csv', index = False) 

#### 100 pct

In [21]:
# 100 pct
path = "/home/lola/math_cluster/output/output-lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765/lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765.output_events.xml.gz"
stuck_veh = countStuckVehicles(path)
stuck_veh_100pct = pd.DataFrame({'n_stuck_veh': stuck_veh,'sample_size': "100-pct", 'alpha': 1.0,
                                'global_seed': "default", 'stuck_time': 30.0 }, index = [0])

#### pd.concat and write csv

In [22]:
stuck_veh_100pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_100_pct.csv', index = False) 

In [52]:
# fix naming error 
stuck_time_violation_1pct = pd.read_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_1_pct.csv')
#stuck_time_violation_1pct.rename(columns={"n_stucktime_v":"n_stuck_veh"}, inplace= True)

In [54]:
stuck_time_violation_5pct = pd.read_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_5_pct.csv")
stuck_time_violation_10pct = pd.read_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_10_pct.csv")
stuck_time_violation_25pct = pd.read_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_25_pct.csv")
stuck_time_violation_50pct = pd.read_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_50_pct.csv")
stuck_time_violation_100pct = pd.read_csv("/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_100_pct.csv")

In [56]:
# concat
stuck_time_violation_1_100 = pd.concat([stuck_time_violation_1pct, stuck_time_violation_5pct, stuck_time_violation_10pct,
                                        stuck_time_violation_25pct, stuck_time_violation_50pct, stuck_veh_100pct])

In [58]:
# write csv
stuck_time_violation_1_100.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/stuck_time_violations_1_to_100pct_correct2.csv', index = False) 

## 3. Count number of link leave events

#### Function to filter output_events and count the number of link leave events

In [2]:
# number of link leave events
# Number of stuck vehicles
def countLinkLeaveEvents(pathToEvents):
    events_file = pathToEvents

    # Read events - filter and return the listed event types only
    events = matsim.event_reader(
        events_file,
        types="left link",
    )

    # Save drt_requests by person_id
    link_counts = defaultdict(int)

    # Loop on all filtered events
    for event in events:
        if event["type"] == "left link":
            link_counts[event["link"]] += 1
    
    count = 0 
    for key in link_counts.keys():
        count+= link_counts[key]
    
    return count

#### 1 pct

In [ ]:
# Count for each case
link_leave_1pct = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1pct-" + str(elem) + "-fCf_sCF_0.01_gS_default_3765/lausitz-1pct-"+str(elem)+ "-fCf_sCf_0.01_gS_default_3765.output_events.xml.gz"
    link_leave = countLinkLeaveEvents(path)
    link_leave_1pct.append(link_leave)
link_leave_1pct = pd.DataFrame({'n_link_leave': link_leave_1pct})
link_leave_1pct.insert(1, 'sample_size', "1-pct")
link_leave_1pct.insert(2, 'alpha', 1.0)
link_leave_1pct.insert(3, 'global_seed', "default")
link_leave_1pct.insert(4, 'stuck_time', 30.0)

In [ ]:
link_leave_1pct_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1pct-" + str(elem) + "-fCf_0.01_sCF_0.03162_gS_default_3765/lausitz-1pct-" +str(elem)+ "-fCf_0.01_sCf_0.03162_gS_default_3765.output_events.xml.gz"
    link_leave = countLinkLeaveEvents(path)
    link_leave_1pct_sCf.append(link_leave)
link_leave_1pct_sCf = pd.DataFrame({'n_link_leave': link_leave_1pct_sCf})
link_leave_1pct_sCf.insert(1, 'sample_size', "1-pct")
link_leave_1pct_sCf.insert(2, 'alpha', 0.75)
link_leave_1pct_sCf.insert(3, 'global_seed', "default")
link_leave_1pct_sCf.insert(4, 'stuck_time', 30.0)

In [ ]:
#  1 pct random seed
link_leave_1pct_rGs = pd.DataFrame()
rGs = [ 4711,3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]
for seed in rGs:
    if (seed == 4711):
        global_seed = "rnd_" + str(seed)
        # insert number of stuck time violations from the first 1 pct sample
        temp = {'n_link_leave': link_leave_1pct["n_link_leave"].iloc[0], 'sample_size': '1-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': global_seed  }
        temp = pd.DataFrame(data=temp, index=[rGs.index(4711)])
        link_leave_1pct_rGs = pd.concat([link_leave_1pct_rGs, temp])
    elif (seed == 3254):
        path ="/home/lola/math_cluster/output/output-lausitz-1pct-1-fCf_sCF_0.01_gS_3254_3765/lausitz-1pct-1-fCf_sCf_0.01_gS_3254_3765.output_events.xml.gz"
        link_leave = countLinkLeaveEvents(path)
        temp2 = pd.DataFrame({'n_link_leave': link_leave, 'sample_size': '1-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': "rnd_" + str(seed)  }, index = [rGs.index(3254)])
        link_leave_1pct_rGs = pd.concat([link_leave_1pct_rGs, temp2], axis = 0)
    else:
        path = "/home/lola/math_cluster/output/output-lausitz-1.0-pct-1-fCf_sCF_0.01_gS_" +str(seed) + "_3765/lausitz-1.0-pct-1-fCf_sCF_0.01_gS_" + str(seed) + "_3765.output_events.xml.gz"
        stuck_veh = countLinkLeaveEvents(path)
        temp2 = pd.DataFrame({'n_link_leave': stuck_veh, 'sample_size': '1-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': "rnd_" + str(seed)  }, index = [rGs.index(seed)] )
        link_leave_1pct_rGs = pd.concat([link_leave_1pct_rGs, temp2], axis = 0)

In [ ]:
# 1 pct sT, alpha = 1
link_leave_1pct_sT = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-" + str(elem) + "-fCf_sCF_0.01_gS_4711_sT_3000.0_3765/lausitz-1-pct-" + str(elem) + "-fCf_sCF_0.01_gS_4711_sT_3000.0_3765.output_events.xml.gz"
    link_leave = countLinkLeaveEvents(path)
    link_leave_1pct_sT.append(link_leave)
link_leave_1pct_sT = pd.DataFrame({'n_link_leave': link_leave_1pct_sT})
link_leave_1pct_sT.insert(1, 'sample_size', "1-pct")
link_leave_1pct_sT.insert(2, 'alpha', 1.0)
link_leave_1pct_sT.insert(3, 'global_seed', "default")
link_leave_1pct_sT.insert(4, 'stuck_time', 3000.0)

In [ ]:
# 1 pct, sT scaled, alpha = 0.75
link_leave_1pct_sT_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-" + str(elem) + "-fCf_0.01_sCF_0.03162_gS_4711_sT_3000.0_3765/lausitz-1-pct-" + str(elem) +"-fCf_0.01_sCF_0.03162_gS_4711_sT_3000.0_3765.output_events.xml.gz"
    link_leave = countLinkLeaveEvents(path)
    link_leave_1pct_sT_sCf.append(link_leave)
link_leave_1pct_sT_sCf = pd.DataFrame({'n_link_leave': link_leave_1pct_sT_sCf})
link_leave_1pct_sT_sCf.insert(1, 'sample_size', "1-pct")
link_leave_1pct_sT_sCf.insert(2, 'alpha', 0.75)
link_leave_1pct_sT_sCf.insert(3, 'global_seed', "default")
link_leave_1pct_sT_sCf.insert(4, 'stuck_time', 3000.0)

In [ ]:
# concat 1 pct 
link_leave_1pct_all = pd.concat([link_leave_1pct, link_leave_1pct_sCf, link_leave_1pct_rGs, link_leave_1pct_sT, link_leave_1pct_sT_sCf], axis= 0)

In [13]:
# write 1 pct to csv
link_leave_1pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/link_leave_1pct_all.csv', index = False) 

#### 5 pct

In [ ]:
link_leave_5pct = []
for elem in range(1,11,1):
    if (elem==6):
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765/lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765-2.output_events.xml.gz"
        link_leave = countLinkLeaveEvents(path)
        link_leave_5pct.append(link_leave)
    else:
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(elem) + "-fCf_sCF_0.05_gS_4711_3765/lausitz-5.0-pct-"+str(elem)+ "-fCf_sCF_0.05_gS_4711_3765.output_events.xml.gz"
        link_leave = countLinkLeaveEvents(path)
        link_leave_5pct.append(link_leave)
link_leave_5pct = pd.DataFrame({'n_link_leave': link_leave_5pct})
link_leave_5pct.insert(1, 'sample_size', "5-pct")
link_leave_5pct.insert(2, 'alpha', 1.0)
link_leave_5pct.insert(3, 'global_seed', "default")
link_leave_5pct.insert(4, 'stuck_time', 30.0)

In [ ]:
link_leave_5pct_sCf = []
for elem in range(1,11,1):
    if (elem==6):
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-6-fCf_0.05_sCF_0.10574_gS_4711_3765/lausitz-5.0-pct-6-fCf_0.05_sCF_0.10574_gS_4711_3765-2.output_events.xml.gz"
        link_leave = countLinkLeaveEvents(path)
        link_leave_5pct_sCf.append(link_leave)
    else:
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(elem) + "-fCf_0.05_sCF_0.10574_gS_4711_3765/lausitz-5.0-pct-"+str(elem)+ "-fCf_0.05_sCF_0.10574_gS_4711_3765.output_events.xml.gz"
        link_leave = countLinkLeaveEvents(path)
        link_leave_5pct_sCf.append(link_leave)
link_leave_5pct_sCf = pd.DataFrame({'n_link_leave': link_leave_5pct_sCf})
link_leave_5pct_sCf.insert(1, 'sample_size', "5-pct")
link_leave_5pct_sCf.insert(2, 'alpha', 0.75)
link_leave_5pct_sCf.insert(3, 'global_seed', "default")
link_leave_5pct_sCf.insert(4, 'stuck_time', 30.0)



In [20]:
# 5 pct random global seed, alpha = 1
link_leave_5pct_rGs = pd.DataFrame()
rGs = [4711, 3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]

for seed in rGs:
    if (seed ==4711):
        global_seed = "rnd_" + str(seed)
        temp = {'n_link_leave': link_leave_5pct["n_link_leave"].iloc[0], 'sample_size': '5-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': global_seed  }
        temp = pd.DataFrame(data=temp, index=[rGs.index(4711)])
        link_leave_5pct_rGs = pd.concat([link_leave_5pct_rGs, temp])
    else:
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-1-fCf_sCF_0.05_gS_" + str(seed) + "_3765/lausitz-5.0-pct-1-fCf_sCF_0.05_gS_"+ str(seed) +"_3765.output_events.xml.gz"
        link_leave = countLinkLeaveEvents(path)
        temp2 = pd.DataFrame({'n_link_leave': link_leave, 'sample_size': '5-pct', 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': "rnd_" + str(seed)  }, index = [rGs.index(seed)])
        link_leave_5pct_rGs = pd.concat([link_leave_5pct_rGs, temp2], axis = 0)



In [21]:
# 5 pct, alpha = 1.0, sT scaled
link_leave_5pct_sT = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-" + str(elem) + "-fCf_sCF_0.05_gS_4711_sT_600.0_3765/lausitz-5-pct-" + str(elem) +"-fCf_sCF_0.05_gS_4711_sT_600.0_3765.output_events.xml.gz"
    link_leave = countLinkLeaveEvents(path)
    link_leave_5pct_sT.append(link_leave)
link_leave_5pct_sT = pd.DataFrame({'n_link_leave': link_leave_5pct_sT})

link_leave_5pct_sT.insert(1, 'sample_size', "5-pct")
link_leave_5pct_sT.insert(2, 'alpha', 1.0)
link_leave_5pct_sT.insert(3, 'global_seed', "default")
link_leave_5pct_sT.insert(4, 'stuck_time', 600.0)

In [22]:
# 5 pct, sT scaled, alpha = 0.75
# 5 pct, alpha = 0.75, sT scaled
link_leave_5pct_sT_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-" + str (elem) + "-fCf_0.05_sCF_0.10574_gS_4711_sT_600.0_3765/lausitz-5-pct-" + str(elem) + "-fCf_0.05_sCF_0.10574_gS_4711_sT_600.0_3765.output_events.xml.gz"
    link_leave = countLinkLeaveEvents(path)
    link_leave_5pct_sT_sCf.append(link_leave)
link_leave_5pct_sT_sCf = pd.DataFrame({'n_link_leave': link_leave_5pct_sT_sCf})

link_leave_5pct_sT_sCf.insert(1, 'sample_size', "5-pct")
link_leave_5pct_sT_sCf.insert(2, 'alpha', 0.75)
link_leave_5pct_sT_sCf.insert(3, 'global_seed', "default")
link_leave_5pct_sT_sCf.insert(4, 'stuck_time', 600.0)

In [23]:
link_leave_5pct_all = pd.concat([link_leave_5pct, link_leave_5pct_sCf, link_leave_5pct_rGs, link_leave_5pct_sT, link_leave_5pct_sT_sCf], axis = 0)

In [24]:
link_leave_5pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/link_leave_5pct_all.csv', index = False) 

#### 10 pct

In [ ]:
link_leave_10pct = []
for elem in range(1,11,1):
    #       /home/lola/math_cluster/output/output-lausitz-10.0-pct-1-fCf_sCF_0.1_gS_4711_3765/lausitz-10.0-pct-1-fCf_sCF_0.1_gS_4711_3765.output_events.xml.gz
    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" + str(elem) + "-fCf_sCF_0.1_gS_4711_3765/lausitz-10.0-pct-"+str(elem)+ "-fCf_sCF_0.1_gS_4711_3765.output_events.xml.gz"
    link_leave = countLinkLeaveEvents(path)
    link_leave_10pct.append(link_leave)
link_leave_10pct = pd.DataFrame({'n_link_leave': link_leave_10pct})
link_leave_10pct.insert(1, 'sample_size', "10-pct")
link_leave_10pct.insert(2, 'alpha', 1.0)
link_leave_10pct.insert(3, 'global_seed', "default")
link_leave_10pct.insert(4, 'stuck_time', 30.0)



In [ ]:
link_leave_10pct_sCf = []
for elem in range(1,11,1):
    #       /home/lola/math_cluster/output/output-lausitz-10.0-pct-1-fCf_0.1_sCF_0.17783_gS_4711_3765/lausitz-10.0-pct-1-fCf_0.1_sCF_0.17783_gS_4711_3765.output_events.xml.gz     
    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" + str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_3765/lausitz-10.0-pct-" +str(elem)+ "-fCf_0.1_sCF_0.17783_gS_4711_3765.output_events.xml.gz"
    link_leave = countLinkLeaveEvents(path)
    link_leave_10pct_sCf.append(link_leave)
link_leave_10pct_sCf = pd.DataFrame({'n_link_leave': link_leave_10pct_sCf})
link_leave_10pct_sCf.insert(1, 'sample_size', "10-pct")
link_leave_10pct_sCf.insert(2, 'alpha', 0.75)
link_leave_10pct_sCf.insert(3, 'global_seed', "default")
link_leave_10pct_sCf.insert(4, 'stuck_time', 30.0)

In [28]:
# 10 pct, alpha = 1, sT scaled
link_leave_10pct_sT = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-" + str(elem) + "-fCf_sCF_0.1_gS_4711_sT_300.0_3765/lausitz-10-pct-" + str(elem) + "-fCf_sCF_0.1_gS_4711_sT_300.0_3765.output_events.xml.gz"
    link_leave = countLinkLeaveEvents(path)
    link_leave_10pct_sT.append(link_leave)
link_leave_10pct_sT = pd.DataFrame({'n_link_leave': link_leave_10pct_sT})
link_leave_10pct_sT.insert(1, 'sample_size', "10-pct")
link_leave_10pct_sT.insert(2, 'alpha', 1.0)
link_leave_10pct_sT.insert(3, 'global_seed', "default")
link_leave_10pct_sT.insert(4, 'stuck_time', 300.0)

In [29]:
# 10 pct, alpha = 0.75, sT scaled
link_leave_10pct_sT_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-" + str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_sT_300.0_3765/lausitz-10-pct-" + str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_sT_300.0_3765.output_events.xml.gz"
    link_leave = countLinkLeaveEvents(path)
    link_leave_10pct_sT_sCf.append(link_leave)
link_leave_10pct_sT_sCf = pd.DataFrame({'n_link_leave': link_leave_10pct_sT_sCf})
link_leave_10pct_sT_sCf.insert(1, 'sample_size', "10-pct")
link_leave_10pct_sT_sCf.insert(2, 'alpha', 0.75)
link_leave_10pct_sT_sCf.insert(3, 'global_seed', "default")
link_leave_10pct_sT_sCf.insert(4, 'stuck_time', 300.0)

In [30]:
link_leave_10_pct_all = pd.concat([link_leave_10pct, link_leave_10pct_sCf, link_leave_10pct_sT, link_leave_10pct_sT_sCf], axis = 0)

In [31]:
# write csv
link_leave_10_pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/link_leave_10_pct_all.csv', index = False) 

#### 25 pct 

In [33]:
# 25 pct, alpha = 1
path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765/lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765.output_events.xml.gz"
link_leave = countLinkLeaveEvents(path)
link_leave_25pct = pd.DataFrame({'n_link_leave': link_leave,'sample_size': "25-pct", 'alpha': 1.0,
                                'global_seed': "default", 'stuck_time': 30.0 }, index= [0])




In [34]:
# 25 pct, alpha = 0.75
path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765/lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765.output_events.xml.gz"
link_leave = countLinkLeaveEvents(path)
link_leave_25pct_sCf = pd.DataFrame({'n_link_leave': link_leave,'sample_size': "25-pct", 'alpha': 0.75,
                                'global_seed': "default", 'stuck_time': 30.0 }, index = [1])


In [35]:
# 25 pct, alpha = 1, sT scaled
path = "/home/lola/math_cluster/output/output-lausitz-25-pct-1-fCf_sCF_0.25_gS_4711_sT_120.0_3765/lausitz-25-pct-1-fCf_sCF_0.25_gS_4711_sT_120.0_3765.output_events.xml.gz"
link_leave = countLinkLeaveEvents(path)
link_leave_25pct_sT = pd.DataFrame({'n_link_leave': link_leave,'sample_size': "25-pct", 'alpha': 1.0,
                                'global_seed': "default", 'stuck_time': 120.0 }, index = [2])

In [36]:
# 25 pct, alpha = 0.75, sT scaled
path = "/home/lola/math_cluster/output/output-lausitz-25-pct-1-fCf_0.25_sCF_0.35355_gS_4711_sT_120.0_3765/lausitz-25-pct-1-fCf_0.25_sCF_0.35355_gS_4711_sT_120.0_3765.output_events.xml.gz"
link_leave = countLinkLeaveEvents(path)
link_leave_25pct_sT_sCf = pd.DataFrame({'n_link_leave': link_leave,'sample_size': "25-pct", 'alpha': 0.75,
                                'global_seed': "default", 'stuck_time': 120.0 }, index = [3])

In [37]:
link_leave_25_pct_all = pd.concat([link_leave_25pct, link_leave_25pct_sCf, link_leave_25pct_sT, link_leave_25pct_sT_sCf], axis = 0)

In [38]:
# write csv
link_leave_25_pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/link_leave_25_pct_all.csv', index = False) 

#### 50 pct

In [39]:
# 50 pct, alpha = 1
path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765/lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765.output_events.xml.gz"
link_leave = countLinkLeaveEvents(path)
link_leave_50pct = pd.DataFrame({'n_link_leave': link_leave,'sample_size': "50-pct", 'alpha': 1.0,
                                'global_seed': "default", 'stuck_time': 30.0 }, index = [0])


In [40]:
# 50 pct, alpha = 0.75
path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_0.5_sCF_0.5946_gS_4711_3765/lausitz-50.0-pct-fCf_0.5_sCF_0.5946_gS_4711_3765.output_events.xml.gz"
link_leave = countLinkLeaveEvents(path)
link_leave_50pct_sCf = pd.DataFrame({'n_link_leave': link_leave,'sample_size': "50-pct", 'alpha': 0.75,
                                'global_seed': "default", 'stuck_time': 30.0 }, index = [1])

In [41]:
# 50 pct, alpha = 1, sT scaled
path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_sCF_0.5_gS_4711_sT_60.0_3765/lausitz-50-pct-1-fCf_sCF_0.5_gS_4711_sT_60.0_3765.output_events.xml.gz"
link_leave = countLinkLeaveEvents(path)
link_leave_50pct_sT = pd.DataFrame({'n_link_leave': link_leave,'sample_size': "50-pct", 'alpha': 1.0,
                                'global_seed': "default", 'stuck_time': 60.0 }, index = [2])

In [42]:
# 50 pct, alpha = 0.75, sT scaled
path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_0.5_sCF_0.5946_gS_4711_sT_60.0_3765/lausitz-50-pct-1-fCf_0.5_sCF_0.5946_gS_4711_sT_60.0_3765.output_events.xml.gz"
link_leave = countLinkLeaveEvents(path)
link_leave_50pct_sT_sCf = pd.DataFrame({'n_link_leave': link_leave,'sample_size': "50-pct", 'alpha': 0.75,
                                'global_seed': "default", 'stuck_time': 60.0 }, index = [3])

In [43]:
link_leave_50_pct_all = pd.concat([link_leave_50pct, link_leave_50pct_sCf, link_leave_50pct_sT, link_leave_50pct_sT_sCf], axis = 0)

In [44]:
# write csv
link_leave_50_pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/link_leave_50_pct_all.csv', index = False) 

#### 100 pct

In [45]:
# 100 pct
path = "/home/lola/math_cluster/output/output-lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765/lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765.output_events.xml.gz"
link_leave = countLinkLeaveEvents(path)
link_leave_100pct = pd.DataFrame({'n_link_leave': link_leave,'sample_size': "100-pct", 'alpha': 1.0,
                                'global_seed': "default", 'stuck_time': 30.0 }, index = [0])

In [46]:
# write csv
link_leave_100pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/link_leave_only_100pct.csv', index = False) 

In [50]:
link_leave_1pct_all = pd.read_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/link_leave_1pct_all.csv')


In [52]:
link_leave_1pct_all.rename(columns={"n_linke_leave": "n_link_leave"}, inplace = True )

In [136]:
link_leave_5pct_all = pd.read_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/link_leave_5pct_all.csv')


In [137]:
toFix = link_leave_5pct_all.iloc[10:20,]

In [138]:
values = toFix["n_linke_leave"].copy()


In [139]:
df = toFix.drop('n_linke_leave', axis=1)

In [140]:
df = df.drop('n_link_leave', axis=1)

In [141]:
df["n_link_leave"] = values

In [142]:
old_part1 = link_leave_5pct_all.iloc[0:10,]

In [143]:
old_part1 = old_part1.drop('n_linke_leave', axis=1)

In [144]:
old_part2 = link_leave_5pct_all.iloc[20:50,]
old_part2 = old_part2.drop('n_linke_leave', axis=1)

In [145]:
link_leave_5pct_all = pd.concat([old_part1, df, old_part2 ], axis = 0)

In [146]:
print(link_leave_5pct_all)

    n_link_leave sample_size  alpha global_seed  stuck_time
0      4448932.0       5-pct   1.00     default        30.0
1      4374804.0       5-pct   1.00     default        30.0
2      4342905.0       5-pct   1.00     default        30.0
3      4452041.0       5-pct   1.00     default        30.0
4      4376599.0       5-pct   1.00     default        30.0
5      4440020.0       5-pct   1.00     default        30.0
6      4445793.0       5-pct   1.00     default        30.0
7      4508651.0       5-pct   1.00     default        30.0
8      4442686.0       5-pct   1.00     default        30.0
9      4404193.0       5-pct   1.00     default        30.0
10     4443451.0       5-pct   0.75     default        30.0
11     4373339.0       5-pct   0.75     default        30.0
12     4340804.0       5-pct   0.75     default        30.0
13     4448246.0       5-pct   0.75     default        30.0
14     4375607.0       5-pct   0.75     default        30.0
15     4443274.0       5-pct   0.75     

In [147]:
link_leave_10_pct_all = pd.read_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/link_leave_10_pct_all.csv')


In [148]:
link_leave_25_pct_all = pd.read_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/link_leave_25_pct_all.csv')


In [149]:
link_leave_50_pct_all = pd.read_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/link_leave_50_pct_all.csv')


In [150]:
link_leave_100pct = pd.read_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/link_leave_only_100pct.csv')

In [151]:
link_leave_1_100 = pd.concat([link_leave_1pct_all, link_leave_5pct_all, link_leave_10_pct_all, link_leave_25_pct_all, link_leave_50_pct_all, link_leave_100pct ], axis = 0)

In [156]:
# write csv
link_leave_1_100.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/link_leave_1_100pct.csv', index = False) 

## 4. Executed Scores at iteration 500

#### 1 pct

In [35]:
#### scores ####
# 1pct, alpha = 1
scores_1pct =pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1pct-" + str(elem) + "-fCf_sCF_0.01_gS_default_3765/lausitz-1pct-" + str(elem) + "-fCf_sCf_0.01_gS_default_3765.scorestats.csv"
    temp = pd.read_csv(path, sep=";")
    df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "1-pct", "sample_nr": elem, "alpha": 1.0, "stuck_time": 30.0, "global_seed": "default" }, index = [elem])
    scores_1pct = pd.concat([scores_1pct, df], axis = 0)
    

In [36]:
# 1 pct, alpha = 0.75
scores_1pct_sCf = pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1pct-" + str(elem) + "-fCf_0.01_sCF_0.03162_gS_default_3765/lausitz-1pct-" + str(elem) + "-fCf_0.01_sCf_0.03162_gS_default_3765.scorestats.csv"
    temp = pd.read_csv(path, sep=";")
    df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "1-pct", "sample_nr": elem, "alpha": 0.75, "stuck_time": 30.0, "global_seed": "default" }, index = [elem])
    scores_1pct_sCf = pd.concat([scores_1pct_sCf, df], axis = 0)

In [37]:
# 1 pct, alpha 1, random global seed
#  1 pct random seed
scores_1pct_rGs = pd.DataFrame()
rGs = [ 4711,3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]
for seed in rGs:
    if (seed == 4711):
        global_seed = "rnd_" + str(seed)
        # insert number of stuck time violations from the first 1 pct sample
        df = pd.DataFrame({"avg_executed_it_500":scores_1pct["avg_executed_it_500"].iloc[0], "sample_size": "1-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        scores_1pct_rGs = pd.concat([scores_1pct_rGs, df])
    elif (seed == 3254):
        global_seed = "rnd_" + str(seed)
        path ="/home/lola/math_cluster/output/output-lausitz-1pct-1-fCf_sCF_0.01_gS_3254_3765/lausitz-1pct-1-fCf_sCf_0.01_gS_3254_3765.scorestats.csv"
        temp = pd.read_csv(path, sep = ";")
        df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "1-pct", "sample_nr": 1, "alpha":1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        scores_1pct_rGs = pd.concat([scores_1pct_rGs, df], axis = 0)
    else:
        global_seed = "rnd_" + str(seed)
        path = "/home/lola/math_cluster/output/output-lausitz-1.0-pct-1-fCf_sCF_0.01_gS_" + str(seed) + "_3765/lausitz-1.0-pct-1-fCf_sCF_0.01_gS_" + str(seed) + "_3765.scorestats.csv"
        temp = pd.read_csv(path, sep = ";")
        df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "1-pct", "sample_nr": 1, "alpha":1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        scores_1pct_rGs = pd.concat([scores_1pct_rGs, df], axis = 0)

In [38]:
# 1 pct, alpha = 1, sT scaled
scores_1pct_sT =pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-" + str(elem) + "-fCf_sCF_0.01_gS_4711_sT_3000.0_3765/lausitz-1-pct-" + str(elem) + "-fCf_sCF_0.01_gS_4711_sT_3000.0_3765.scorestats.csv"
    temp = pd.read_csv(path, sep=";")
    df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "1-pct", "sample_nr": elem, "alpha": 1.0, "stuck_time": 3000.0, "global_seed": "default" }, index = [elem])
    scores_1pct_sT = pd.concat([scores_1pct_sT, df], axis = 0)


In [39]:
# 1 pct, alpha = 0.75, sT scaled
scores_1pct_sT_sCf =pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-"+ str(elem) + "-fCf_0.01_sCF_0.03162_gS_4711_sT_3000.0_3765/lausitz-1-pct-" + str(elem) + "-fCf_0.01_sCF_0.03162_gS_4711_sT_3000.0_3765.scorestats.csv"
    temp = pd.read_csv(path, sep=";")
    df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "1-pct", "sample_nr": elem, "alpha": 0.75, "stuck_time": 3000.0, "global_seed": "default" }, index = [elem])
    scores_1pct_sT_sCf = pd.concat([scores_1pct_sT_sCf, df], axis = 0)

In [40]:
scores_1pct_all = pd.concat([scores_1pct, scores_1pct_sCf, scores_1pct_rGs, scores_1pct_sT, scores_1pct_sT_sCf])


In [41]:
scores_1pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/scores_all_1pct_samples.csv', index = False) 

#### 5 pct 

In [42]:
#### scores, 5 pct ####
# 5pct, alpha = 1
scores_5pct = pd.DataFrame()

for elem in range(1,11,1):
    if (elem==6):
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765/lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765-2.scorestats.csv"
        temp = pd.read_csv(path, sep=";")
        df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "5-pct", "sample_nr": elem, "alpha": 1.0, "stuck_time": 30.0, "global_seed": "default" }, index = [elem])
        scores_5pct = pd.concat([scores_5pct, df], axis = 0)
    else:
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(elem) + "-fCf_sCF_0.05_gS_4711_3765/lausitz-5.0-pct-" + str(elem) + "-fCf_sCF_0.05_gS_4711_3765.scorestats.csv"
        temp = pd.read_csv(path, sep=";")
        df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "5-pct", "sample_nr": elem, "alpha": 1.0, "stuck_time": 30.0, "global_seed": "default" }, index = [elem])
        scores_5pct = pd.concat([scores_5pct, df], axis = 0)


In [43]:
# 5 pct, alpha 0.75 
scores_5pct_sCf = pd.DataFrame()

for elem in range(1,11,1):
    if (elem==6):
        path ="/home/lola/math_cluster/output/output-lausitz-5.0-pct-6-fCf_0.05_sCF_0.10574_gS_4711_3765/lausitz-5.0-pct-6-fCf_0.05_sCF_0.10574_gS_4711_3765-2.scorestats.csv"
        temp = pd.read_csv(path, sep=";")
        df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "5-pct", "sample_nr": elem, "alpha": 0.75, "stuck_time": 30.0, "global_seed": "default" }, index = [elem])
        scores_5pct_sCf = pd.concat([scores_5pct_sCf, df], axis = 0)

    else:
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(elem) + "-fCf_0.05_sCF_0.10574_gS_4711_3765/lausitz-5.0-pct-" + str(elem) + "-fCf_0.05_sCF_0.10574_gS_4711_3765.scorestats.csv"
        temp = pd.read_csv(path, sep=";")
        df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "5-pct", "sample_nr": elem, "alpha":0.75, "stuck_time": 30.0, "global_seed": "default" }, index = [elem])
        scores_5pct_sCf = pd.concat([scores_5pct_sCf, df], axis = 0)

In [44]:
scores_5pct_rGs = pd.DataFrame()
rGs = [4711, 3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]

for seed in rGs:
    if (seed ==4711):
        global_seed = "rnd_" + str(seed)
        temp = {'avg_executed_it_500': scores_5pct["avg_executed_it_500"].iloc[0], 'sample_size': '5-pct', "sample_nr": 1, 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': global_seed  }
        temp = pd.DataFrame(data=temp, index=[rGs.index(4711)])
        scores_5pct_rGs = pd.concat([scores_5pct_rGs, temp])
    else:
        global_seed = "rnd_" + str(seed)
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-1-fCf_sCF_0.05_gS_" + str(seed) + "_3765/lausitz-5.0-pct-1-fCf_sCF_0.05_gS_" + str(seed) + "_3765.scorestats.csv"
        temp = pd.read_csv(path, sep = ";")
        temp = {'avg_executed_it_500': temp["avg_executed"].iloc[500], 'sample_size': '5-pct', "sample_nr": 1, 'alpha': 1.0, 'stuck_time': 30.0, 'global_seed': global_seed  }
        df = pd.DataFrame(data=temp, index=[rGs.index(seed)])
        scores_5pct_rGs = pd.concat([scores_5pct_rGs, df], axis = 0)


In [45]:
# 5 pct, alpha 1.0, sT scaled
scores_5pct_sT =pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-" + str(elem) + "-fCf_sCF_0.05_gS_4711_sT_600.0_3765/lausitz-5-pct-" + str(elem) + "-fCf_sCF_0.05_gS_4711_sT_600.0_3765.scorestats.csv"
    temp = pd.read_csv(path, sep=";")
    df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "5-pct", "sample_nr": elem, "alpha": 1.0, "stuck_time": 600.0, "global_seed": "default" }, index = [elem])
    scores_5pct_sT = pd.concat([scores_5pct_sT, df], axis = 0)




In [46]:
# 5 pct, alpha 0.75, sT scaled
scores_5pct_sT_sCf =pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-" + str(elem) + "-fCf_0.05_sCF_0.10574_gS_4711_sT_600.0_3765/lausitz-5-pct-" + str(elem) + "-fCf_0.05_sCF_0.10574_gS_4711_sT_600.0_3765.scorestats.csv"
    temp = pd.read_csv(path, sep=";")
    df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "5-pct", "sample_nr": elem, "alpha": 0.75, "stuck_time": 600.0, "global_seed": "default" }, index = [elem])
    scores_5pct_sT_sCf = pd.concat([scores_5pct_sT_sCf, df], axis = 0)

  

In [47]:
scores_5pct_all = pd.concat([scores_5pct, scores_5pct_sCf, scores_5pct_rGs, scores_5pct_sT, scores_5pct_sT_sCf])


In [48]:
scores_5pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/scores_all_5pct_samples.csv', index = False) 

#### 10 pct

In [49]:
#### scores, 10 pct ####
# 10 pct, alpha = 1.0
scores_10pct = pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" + str(elem) + "-fCf_sCF_0.1_gS_4711_3765/lausitz-10.0-pct-" + str(elem) + "-fCf_sCF_0.1_gS_4711_3765.scorestats.csv"
    temp = pd.read_csv(path, sep=";")
    df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "10-pct", "sample_nr": elem, "alpha": 1.0, "stuck_time": 30.0, "global_seed": "default" }, index = [elem])
    scores_10pct = pd.concat([scores_10pct, df], axis = 0)


In [50]:
# 10 pct, alpha = 0.75
scores_10pct_sCf = pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" + str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_3765/lausitz-10.0-pct-" + str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_3765.scorestats.csv"
    temp = pd.read_csv(path, sep=";")
    df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "10-pct", "sample_nr": elem, "alpha": 0.75, "stuck_time": 30.0, "global_seed": "default" }, index = [elem])
    scores_10pct = pd.concat([scores_10pct, df], axis = 0)

In [51]:
# 10 pct, alpha = 1, sT scaled
scores_10pct_sT = pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-" + str(elem) + "-fCf_sCF_0.1_gS_4711_sT_300.0_3765/lausitz-10-pct-" + str(elem) + "-fCf_sCF_0.1_gS_4711_sT_300.0_3765.scorestats.csv"
    temp = pd.read_csv(path, sep=";")
    df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "10-pct", "sample_nr": elem, "alpha": 1.0, "stuck_time": 300.0, "global_seed": "default" }, index = [elem])
    scores_10pct_sT = pd.concat([scores_10pct_sT, df], axis = 0)

In [52]:
# 10 pct, alpha = 0.75, sT scaled
# 10 pct, alpha = 1, sT scaled
scores_10pct_sT_sCf = pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-" + str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_sT_300.0_3765/lausitz-10-pct-" + str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_sT_300.0_3765.scorestats.csv"
    temp = pd.read_csv(path, sep=";")
    df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "10-pct", "sample_nr": elem, "alpha": 0.75, "stuck_time": 300.0, "global_seed": "default" }, index = [elem])
    scores_10pct_sT_sCf = pd.concat([scores_10pct_sT_sCf, df], axis = 0)


In [53]:
scores_10pct_all = pd.concat([scores_10pct, scores_10pct_sCf, scores_10pct_sT, scores_10pct_sT_sCf])

In [54]:
scores_10pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/scores_all_10pct_samples.csv', index = False) 

#### 25 pct

In [55]:
#### scores, 25 pct ####
scores_25pct = pd.DataFrame()
 # 25 pct, alpha = 1
path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765/lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765.scorestats.csv"
temp = pd.read_csv(path, sep = ";")
df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "25-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": "default" }, index = [0])
scores_25pct = pd.concat([scores_25pct, df], axis = 0)

# 25 pct, alpha = 0.75
path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765/lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765.scorestats.csv"
temp = pd.read_csv(path, sep = ";")
df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "25-pct", "sample_nr": 1, "alpha": 0.75, "stuck_time": 30.0, "global_seed": "default" }, index = [1])
scores_25pct = pd.concat([scores_25pct, df], axis = 0)

 # 25 pct, alpha = 1, sT scaled
path = "/home/lola/math_cluster/output/output-lausitz-25-pct-1-fCf_sCF_0.25_gS_4711_sT_120.0_3765/lausitz-25-pct-1-fCf_sCF_0.25_gS_4711_sT_120.0_3765.scorestats.csv"
temp = pd.read_csv(path, sep = ";")
df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "25-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 120.0, "global_seed": "default" }, index = [2])
scores_25pct = pd.concat([scores_25pct, df], axis = 0)

 # 25 pct, alpha = 0.75, sT scaled
path = "/home/lola/math_cluster/output/output-lausitz-25-pct-1-fCf_0.25_sCF_0.35355_gS_4711_sT_120.0_3765/lausitz-25-pct-1-fCf_0.25_sCF_0.35355_gS_4711_sT_120.0_3765.scorestats.csv"
temp = pd.read_csv(path, sep = ";")
df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "25-pct", "sample_nr": 1, "alpha": 0.75, "stuck_time": 120.0, "global_seed": "default" }, index = [0])
scores_25pct = pd.concat([scores_25pct, df], axis = 0)

 

In [56]:
scores_25pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/scores_all_25pct_samples.csv', index = False) 

#### 50 pct

In [57]:
#### scores, 50 pct #### 
scores_50pct = pd.DataFrame()
# 50 pct, alpha = 1
path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765/lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765.scorestats.csv"
temp = pd.read_csv(path, sep = ";")
df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "50-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": "default" }, index = [0])
scores_50pct = pd.concat([scores_50pct, df], axis = 0)

# 50 pct, alpha = 0.75
path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_0.5_sCF_0.5946_gS_4711_3765/lausitz-50.0-pct-fCf_0.5_sCF_0.5946_gS_4711_3765.scorestats.csv"
temp = pd.read_csv(path, sep = ";")
df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "50-pct", "sample_nr": 1, "alpha": 0.75, "stuck_time": 30.0, "global_seed": "default" }, index = [1])
scores_50pct = pd.concat([scores_50pct, df], axis = 0)

# 50 pct/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_sCF_0.5_gS_4711_sT_60.0_3765/lausitz-50-pct-1-fCf_sCF_0.5_gS_4711_sT_60.0_3765.scorestats.csv"
temp = pd.read_csv(path, sep = ";")
df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "50-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 60.0, "global_seed": "default" }, index = [2])
scores_50pct = pd.concat([scores_50pct, df], axis = 0)

# 50 pct, alpha = 0.75, sT scaled
path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_0.5_sCF_0.5946_gS_4711_sT_60.0_3765/lausitz-50-pct-1-fCf_0.5_sCF_0.5946_gS_4711_sT_60.0_3765.scorestats.csv"
temp = pd.read_csv(path, sep = ";")
df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "50-pct", "sample_nr": 1, "alpha": 0.75, "stuck_time": 60.0, "global_seed": "default" }, index = [0])
scores_50pct = pd.concat([scores_50pct, df], axis = 0)


# 25 pct doubled, alpha = 1.0, sT 30.0
path = "/home/lola/Nextcloud/Masterarbeit/03_Outputs_From_RunsLausitz/output-lausitz-25-pct-doubled-fCf_0.5_sCF_0.5_gS_4711_3765/lausitz-25-pct-doubled-fCf_0.5_sCF_0.5_gS_4711__3765.scorestats.csv"
temp = pd.read_csv(path, sep = ";")
df = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "25-pct-doubled", "sample_nr": 1, "alpha": 1.0 , "stuck_time": 30.0, "global_seed": "default" }, index = [0])
scores_50pct = pd.concat([scores_50pct, df], axis = 0)

In [58]:
scores_50pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/scores_all_50pct_samples.csv', index = False)

#### 100 pct

In [59]:
#### scores , 100 pct ####
path = "/home/lola/math_cluster/output/output-lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765/lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765.scorestats.csv"
temp = pd.read_csv(path, sep = ";")
scores_100pct = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "100-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": "default" }, index = [0])

path = "/home/lola/Nextcloud/Masterarbeit/03_Outputs_From_RunsLausitz/output-lausitz-25.0-pct-quadrupled-fCf_1.0_sCF_1.0_gS_4711_3765/lausitz-25-pct-quadrupled-fCf_1.0_sCF_1.0_gS_4711__3765.scorestats.csv"
temp = pd.read_csv(path, sep = ";")
scores_25_pct_quadrupled = pd.DataFrame({"avg_executed_it_500":temp["avg_executed"].iloc[500], "sample_size": "25-pct-quadrupled", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": "default" }, index = [0])

In [60]:
scores_1_100 = pd.concat([scores_1pct_all, scores_5pct_all, scores_10pct_all, scores_25pct, scores_50pct, scores_100pct, scores_25_pct_quadrupled], axis = 0)

In [61]:
scores_1_100.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/scores_all_1_100pct_samples.csv', index = False) 

## 5./6. Average travel time and traveled distance

#### 1 pct

In [35]:
#### average travel time and average traveled distance ####
# trav_time and traveled_distance
# 1 pct, alpha = 1
avg_trav_time_1pct = pd.DataFrame()
avg_trav_dist_1pct = pd.DataFrame()
for elem in range(1,11,1):
    #       /home/lola/math_cluster/output/output-lausitz-1pct-1-fCf_sCF_0.01_gS_default_3765/lausitz-1pct-1-fCf_sCf_0.01_gS_default_3765.output_trips.csv.gz
    path = "/home/lola/math_cluster/output/output-lausitz-1pct-"+ str(elem) +"-fCf_sCF_0.01_gS_default_3765/lausitz-1pct-"+ str(elem) +"-fCf_sCf_0.01_gS_default_3765.output_trips.csv.gz"
    temp = pd.read_csv(path, compression= "gzip", sep=";")
    temp["trav_time"] = pd.to_timedelta(temp.trav_time)
    avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
    df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "1-pct", "sample_nr": elem, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": "default"}, index=[elem])
    avg_trav_time_1pct = pd.concat([avg_trav_time_1pct, df_avg_trav_time], axis = 0)

    avg_trav_dist = temp["traveled_distance"].mean()
    df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "1-pct", "sample_nr": elem, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": "default"}, index=[elem])
    avg_trav_dist_1pct = pd.concat([avg_trav_dist_1pct, df_avg_trav_dist], axis = 0)

In [36]:
# 1 pct, alpha = 0.75
avg_trav_time_1pct_sCf = pd.DataFrame()
avg_trav_dist_1pct_sCf = pd.DataFrame()
for elem in range(1,11,1):      
    path = "/home/lola/math_cluster/output/output-lausitz-1pct-" +str(elem) + "-fCf_0.01_sCF_0.03162_gS_default_3765/lausitz-1pct-" + str(elem) + "-fCf_0.01_sCf_0.03162_gS_default_3765.output_trips.csv.gz"
    temp = pd.read_csv(path, compression= "gzip", sep=";")
    temp["trav_time"] = pd.to_timedelta(temp.trav_time)
    avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
    df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "1-pct", "sample_nr": elem, "alpha" : 0.75, "stuck_time": 30.0, "global_seed": "default"}, index=[elem])
    avg_trav_time_1pct_sCf = pd.concat([avg_trav_time_1pct_sCf, df_avg_trav_time], axis = 0)

    avg_trav_dist = temp["traveled_distance"].mean()
    df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "1-pct", "sample_nr": elem, "alpha" : 0.75, "stuck_time": 30.0, "global_seed": "default"}, index=[elem])
    avg_trav_dist_1pct_sCf = pd.concat([avg_trav_dist_1pct_sCf, df_avg_trav_dist], axis = 0)

In [37]:
# 1 pct rGs
avg_trav_time_1pct_rGs = pd.DataFrame()
avg_trav_dist_1pct_rGs = pd.DataFrame()
rGs = [ 4711,3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]
for seed in rGs:
    if (seed == 4711):
        global_seed = "rnd_" + str(seed)
        # Create Data frame and insert first value from avg_trav_time_1pct
        df1 = pd.DataFrame({"avg_trav_time":avg_trav_time_1pct["avg_trav_time"].iloc[0], "sample_size": "1-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        avg_trav_time_1pct_rGs = pd.concat([avg_trav_time_1pct_rGs, df1])
        
        # Create Data frame and insert first value from avg_trav_dist_1pct
        df1 = pd.DataFrame({"avg_trav_dist":avg_trav_dist_1pct["avg_trav_dist"].iloc[0], "sample_size": "1-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        avg_trav_dist_1pct_rGs = pd.concat([avg_trav_dist_1pct_rGs, df1])

    elif (seed == 3254):
        global_seed = "rnd_" + str(seed)
        path ="/home/lola/math_cluster/output/output-lausitz-1pct-1-fCf_sCF_0.01_gS_3254_3765/lausitz-1pct-1-fCf_sCf_0.01_gS_3254_3765.output_trips.csv.gz"
        temp = pd.read_csv(path, compression= "gzip", sep=";")
        temp["trav_time"] = pd.to_timedelta(temp.trav_time)
        avg_trav_time =  temp["trav_time"].sum() / temp["trav_time"].shape[0]
        # average travel time
        df1 = pd.DataFrame({"avg_trav_time":avg_trav_time, "sample_size": "1-pct", "sample_nr": 1, "alpha":1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        avg_trav_time_1pct_rGs = pd.concat([avg_trav_time_1pct_rGs, df1], axis = 0)
        # average traveled distance
        avg_trav_dist = temp["traveled_distance"].mean()
        df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "1-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": global_seed}, index=[rGs.index(seed)])
        avg_trav_dist_1pct_rGs = pd.concat([avg_trav_dist_1pct_rGs, df_avg_trav_dist], axis = 0)

    else:
        global_seed = "rnd_" + str(seed)
        path ="/home/lola/math_cluster/output/output-lausitz-1.0-pct-1-fCf_sCF_0.01_gS_"+ str(seed) + "_3765/lausitz-1.0-pct-1-fCf_sCF_0.01_gS_" +str(seed) +"_3765.output_trips.csv.gz"
        temp = pd.read_csv(path, compression= "gzip", sep=";")
        temp["trav_time"] = pd.to_timedelta(temp.trav_time)
        avg_trav_time =  temp["trav_time"].sum() / temp["trav_time"].shape[0]
        # average travel time
        df1 = pd.DataFrame({"avg_trav_time":avg_trav_time, "sample_size": "1-pct", "sample_nr": 1, "alpha":1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        avg_trav_time_1pct_rGs = pd.concat([avg_trav_time_1pct_rGs, df1], axis = 0)
        # average traveled distance
        avg_trav_dist = temp["traveled_distance"].mean()
        df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "1-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": global_seed}, index=[rGs.index(seed)])
        avg_trav_dist_1pct_rGs = pd.concat([avg_trav_dist_1pct_rGs, df_avg_trav_dist], axis = 0)

In [38]:
# 1 pct, alpha = 1, sT
avg_trav_time_1pct_sT = pd.DataFrame()
avg_trav_dist_1pct_sT = pd.DataFrame()
for sampleNr in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-" +str(sampleNr) +"-fCf_sCF_0.01_gS_4711_sT_3000.0_3765/lausitz-1-pct-" +str(sampleNr) + "-fCf_sCF_0.01_gS_4711_sT_3000.0_3765.output_trips.csv.gz"
    temp = pd.read_csv(path, compression= "gzip", sep=";")
    temp["trav_time"] = pd.to_timedelta(temp.trav_time)
    avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
    df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "1-pct", "sample_nr": sampleNr, "alpha" : 1.0, "stuck_time": 3000.0, "global_seed": "default"}, index=[sampleNr])
    avg_trav_time_1pct_sT = pd.concat([avg_trav_time_1pct_sT, df_avg_trav_time], axis = 0)

    avg_trav_dist = temp["traveled_distance"].mean()
    #print("avg_trav_dist", avg_trav_dist)
    df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "1-pct", "sample_nr": sampleNr, "alpha" : 1.0, "stuck_time": 3000.0, "global_seed": "default"}, index=[sampleNr])
    avg_trav_dist_1pct_sT = pd.concat([avg_trav_dist_1pct_sT, df_avg_trav_dist], axis = 0) 

In [ ]:
# 1 pct, alpha = 0.75, sT 
avg_trav_time_1pct_sT_sCf = pd.DataFrame()
avg_trav_dist_1pct_sT_sCf = pd.DataFrame()
for sampleNr in range(1,11,1):      
    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-" + str(sampleNr) + "-fCf_0.01_sCF_0.03162_gS_4711_sT_3000.0_3765/lausitz-1-pct-" + str(sampleNr) + "-fCf_0.01_sCF_0.03162_gS_4711_sT_3000.0_3765.output_trips.csv.gz"
    temp = pd.read_csv(path, compression= "gzip", sep=";")
    temp["trav_time"] = pd.to_timedelta(temp.trav_time)
    avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
    df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "1-pct", "sample_nr": sampleNr, "alpha" : 0.75, "stuck_time": 3000.0, "global_seed": "default"}, index=[sampleNr])
    avg_trav_time_1pct_sT_sCf = pd.concat([avg_trav_time_1pct_sT_sCf, df_avg_trav_time], axis = 0)

    avg_trav_dist = temp["traveled_distance"].mean()
    #print(avg_trav_dist)
    df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "1-pct", "sample_nr": sampleNr, "alpha" : 0.75, "stuck_time": 3000.0, "global_seed": "default"}, index=[elem])
    avg_trav_dist_1pct_sT_sCf = pd.concat([avg_trav_dist_1pct_sT_sCf, df_avg_trav_dist], axis = 0)

In [ ]:
avg_trav_time_1pct_all = pd.concat([avg_trav_time_1pct, avg_trav_time_1pct_sCf, avg_trav_time_1pct_rGs, avg_trav_time_1pct_sT, avg_trav_time_1pct_sT_sCf], axis = 0)
avg_trav_dist_1pct_all = pd.concat([avg_trav_dist_1pct, avg_trav_dist_1pct_sCf, avg_trav_dist_1pct_rGs, avg_trav_dist_1pct_sT, avg_trav_dist_1pct_sT_sCf], axis = 0)

In [41]:
avg_trav_time_1pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_time_all_1pct_samples.csv', index = False) 
avg_trav_dist_1pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_dist_all_1pct_samples.csv', index = False) 

#### 5 pct

In [42]:
# 5 pct, alpha = 1
#### average travel time and average traveled distance ####
# trav_time and traveled_distance
# 5 pct, alpha = 1
avg_trav_time_5pct = pd.DataFrame()
avg_trav_dist_5pct = pd.DataFrame()
for elem in range(1,11,1):
    if (elem == 6):
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765/lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765-2.output_trips.csv.gz"
        temp = pd.read_csv(path, compression = "gzip", sep = ";")
        temp["trav_time"] = pd.to_timedelta(temp.trav_time)
        avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
        df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "5-pct", "sample_nr": elem, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": "default"}, index=[elem])
        avg_trav_time_5pct = pd.concat([avg_trav_time_5pct, df_avg_trav_time], axis = 0)

        avg_trav_dist = temp["traveled_distance"].mean()
        df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "5-pct", "sample_nr": elem, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": "default"}, index=[elem])
        avg_trav_dist_5pct = pd.concat([avg_trav_dist_5pct, df_avg_trav_dist], axis = 0)
    else:     
        path = "/home/lola/math_cluster/output/output-lausitz-5-pct-"+str(elem) + "-fCf_sCF_0.05_gS_4711_sT_600.0_3765/lausitz-5-pct-" +str(elem) + "-fCf_sCF_0.05_gS_4711_sT_600.0_3765.output_trips.csv.gz"
        temp = pd.read_csv(path, compression= "gzip", sep=";")
        temp["trav_time"] = pd.to_timedelta(temp.trav_time)
        avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
        df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "5-pct", "sample_nr": elem, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": "default"}, index=[elem])
        avg_trav_time_5pct = pd.concat([avg_trav_time_5pct, df_avg_trav_time], axis = 0)

        avg_trav_dist = temp["traveled_distance"].mean()
        df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "5-pct", "sample_nr": elem, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": "default"}, index=[elem])
        avg_trav_dist_5pct = pd.concat([avg_trav_dist_5pct, df_avg_trav_dist], axis = 0)

In [43]:
avg_trav_time_5pct_sCf = pd.DataFrame()
avg_trav_dist_5pct_sCf = pd.DataFrame()
for elem in range(1,11,1):
    if (elem == 6):
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-6-fCf_0.05_sCF_0.10574_gS_4711_3765/lausitz-5.0-pct-6-fCf_0.05_sCF_0.10574_gS_4711_3765-2.output_trips.csv.gz"
        temp = pd.read_csv(path, compression = "gzip", sep = ";")
        temp["trav_time"] = pd.to_timedelta(temp.trav_time)
        avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
        df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "5-pct", "sample_nr": elem, "alpha" : 0.75, "stuck_time": 30.0, "global_seed": "default"}, index=[elem])
        avg_trav_time_5pct_sCf = pd.concat([avg_trav_time_5pct_sCf, df_avg_trav_time], axis = 0)

        avg_trav_dist = temp["traveled_distance"].mean()
        df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "5-pct", "sample_nr": elem, "alpha" : 0.75, "stuck_time": 30.0, "global_seed": "default"}, index=[elem])
        avg_trav_dist_5pct_sCf = pd.concat([avg_trav_dist_5pct_sCf, df_avg_trav_dist], axis = 0)
    else:     
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" +str(elem) + "-fCf_0.05_sCF_0.10574_gS_4711_3765/lausitz-5.0-pct-" + str(elem) + "-fCf_0.05_sCF_0.10574_gS_4711_3765.output_trips.csv.gz"
        temp = pd.read_csv(path, compression= "gzip", sep=";")
        temp["trav_time"] = pd.to_timedelta(temp.trav_time)
        avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
        df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "5-pct", "sample_nr": elem, "alpha" : 0.75, "stuck_time": 30.0, "global_seed": "default"}, index=[elem])
        avg_trav_time_5pct = pd.concat([avg_trav_time_5pct, df_avg_trav_time], axis = 0)

        avg_trav_dist = temp["traveled_distance"].mean()
        df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "5-pct", "sample_nr": elem, "alpha" : 0.75, "stuck_time": 30.0, "global_seed": "default"}, index=[elem])
        avg_trav_dist_5pct = pd.concat([avg_trav_dist_5pct, df_avg_trav_dist], axis = 0)

In [ ]:
# 5 pct random global seed
avg_trav_time_5pct_rGs = pd.DataFrame()
avg_trav_dist_5pct_rGs = pd.DataFrame()
rGs = [ 4711,3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]
for seed in rGs:
    if (seed == 4711):
        global_seed = "rnd_" + str(seed)
        # Create Data frame and insert first value from avg_trav_time_5pct
        df1 = pd.DataFrame({"avg_trav_time":avg_trav_time_5pct["avg_trav_time"].iloc[0], "sample_size": "5-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        avg_trav_time_5pct_rGs = pd.concat([avg_trav_time_5pct_rGs, df1])
        
        # Create Data frame and insert first value from avg_trav_dist_5pct
        df1 = pd.DataFrame({"avg_trav_dist":avg_trav_dist_5pct["avg_trav_dist"].iloc[0], "sample_size": "5-pct", "sample_nr": 1, "alpha": 1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        #print(df1["avg_trav_dist"])
        avg_trav_dist_5pct_rGs = pd.concat([avg_trav_dist_5pct_rGs, df1])

    elif (seed == 3254):
        global_seed = "rnd_" + str(seed)
        path ="/home/lola/math_cluster/output/output-lausitz-5.0-pct-1-fCf_sCF_0.05_gS_3254_3765/lausitz-5.0-pct-1-fCf_sCF_0.05_gS_3254_3765.output_trips.csv.gz"
        temp = pd.read_csv(path, compression= "gzip", sep=";")
        temp["trav_time"] = pd.to_timedelta(temp.trav_time)
        avg_trav_time =  temp["trav_time"].sum() / temp["trav_time"].shape[0]
        # average travel time
        df1 = pd.DataFrame({"avg_trav_time":avg_trav_time, "sample_size": "5-pct", "sample_nr": 1, "alpha":1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        avg_trav_time_5pct_rGs = pd.concat([avg_trav_time_5pct_rGs, df1], axis = 0)
        # average traveled distance
        avg_trav_dist = temp["traveled_distance"].mean()
        df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "5-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": global_seed}, index=[rGs.index(seed)])
        #print(df_avg_trav_dist["avg_trav_dist"])
        avg_trav_dist_5pct_rGs = pd.concat([avg_trav_dist_5pct_rGs, df_avg_trav_dist], axis = 0)

    else:
        global_seed = "rnd_" + str(seed)
        path ="/home/lola/math_cluster/output/output-lausitz-5.0-pct-1-fCf_sCF_0.05_gS_" + str(seed) + "_3765/lausitz-5.0-pct-1-fCf_sCF_0.05_gS_" + str(seed) + "_3765.output_trips.csv.gz"
        temp = pd.read_csv(path, compression= "gzip", sep=";")
        temp["trav_time"] = pd.to_timedelta(temp.trav_time)
        avg_trav_time =  temp["trav_time"].sum() / temp["trav_time"].shape[0]
        # average travel time
        df1 = pd.DataFrame({"avg_trav_time":avg_trav_time, "sample_size": "5-pct", "sample_nr": 1, "alpha":1.0, "stuck_time": 30.0, "global_seed": global_seed }, index = [rGs.index(seed)])
        avg_trav_time_5pct_rGs = pd.concat([avg_trav_time_5pct_rGs, df1], axis = 0)
        # average traveled distance
        avg_trav_dist = temp["traveled_distance"].mean()
        df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "5-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": global_seed}, index=[rGs.index(seed)])
        #print(df_avg_trav_dist["avg_trav_dist"])
        avg_trav_dist_5pct_rGs = pd.concat([avg_trav_dist_5pct_rGs, df_avg_trav_dist], axis = 0)


In [45]:
avg_trav_time_5pct_sT = pd.DataFrame()
avg_trav_dist_5pct_sT = pd.DataFrame()
for elem in range(1,11,1):   
    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-" +str(elem) + "-fCf_sCF_0.05_gS_4711_sT_600.0_3765/lausitz-5-pct-" + str(elem) + "-fCf_sCF_0.05_gS_4711_sT_600.0_3765.output_trips.csv.gz"
    temp = pd.read_csv(path, compression= "gzip", sep=";")
    temp["trav_time"] = pd.to_timedelta(temp.trav_time)
    avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
    df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "5-pct", "sample_nr": elem, "alpha" : 1.0, "stuck_time": 600.0, "global_seed": "default"}, index=[elem])
    avg_trav_time_5pct_sT = pd.concat([avg_trav_time_5pct_sT, df_avg_trav_time], axis = 0)

    avg_trav_dist = temp["traveled_distance"].mean()
    df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "5-pct", "sample_nr": elem, "alpha" : 1.0, "stuck_time": 600.0, "global_seed": "default"}, index=[elem])
    avg_trav_dist_5pct_sT = pd.concat([avg_trav_dist_5pct_sT, df_avg_trav_dist], axis = 0)

In [46]:
avg_trav_time_5pct_sT_sCf = pd.DataFrame()
avg_trav_dist_5pct_sT_sCf = pd.DataFrame()
for elem in range(1,11,1):   
    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-" + str(elem) + "-fCf_0.05_sCF_0.10574_gS_4711_sT_600.0_3765/lausitz-5-pct-" + str(elem) + "-fCf_0.05_sCF_0.10574_gS_4711_sT_600.0_3765.output_trips.csv.gz"
    temp = pd.read_csv(path, compression= "gzip", sep=";")
    temp["trav_time"] = pd.to_timedelta(temp.trav_time)
    avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
    df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "5-pct", "sample_nr": elem, "alpha" : 0.75, "stuck_time": 600.0, "global_seed": "default"}, index=[elem])
    avg_trav_time_5pct_sT_sCf = pd.concat([avg_trav_time_5pct_sT_sCf, df_avg_trav_time], axis = 0)

    avg_trav_dist = temp["traveled_distance"].mean()
    df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "5-pct", "sample_nr": elem, "alpha" : 0.75, "stuck_time": 600.0, "global_seed": "default"}, index=[elem])
    avg_trav_dist_5pct_sT_sCf = pd.concat([avg_trav_dist_5pct_sT_sCf, df_avg_trav_dist], axis = 0)

In [47]:
avg_trav_time_5pct_all = pd.concat([avg_trav_time_5pct, avg_trav_time_5pct_sCf, avg_trav_time_5pct_rGs, avg_trav_time_5pct_sT, avg_trav_time_5pct_sT_sCf], axis = 0)

avg_trav_dist_5pct_all = pd.concat([avg_trav_dist_5pct, avg_trav_dist_5pct_sCf, avg_trav_dist_5pct_rGs, avg_trav_dist_5pct_sT, avg_trav_dist_5pct_sT_sCf], axis = 0)

In [48]:
avg_trav_time_5pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_time_all_5pct_samples.csv', index = False) 
avg_trav_dist_5pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_dist_all_5pct_samples.csv', index = False) 

#### 10 pct

In [49]:
avg_trav_time_10pct = pd.DataFrame()
avg_trav_dist_10pct = pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" +str(elem) + "-fCf_sCF_0.1_gS_4711_3765/lausitz-10.0-pct-" +str(elem) + "-fCf_sCF_0.1_gS_4711_3765.output_trips.csv.gz"
    temp = pd.read_csv(path, compression= "gzip", sep=";")
    temp["trav_time"] = pd.to_timedelta(temp.trav_time)
    avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
    df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "10-pct", "sample_nr": elem, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": "default"}, index=[elem])
    avg_trav_time_10pct = pd.concat([avg_trav_time_10pct, df_avg_trav_time], axis = 0)

    avg_trav_dist = temp["traveled_distance"].mean()
    df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "10-pct", "sample_nr": elem, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": "default"}, index=[elem])
    avg_trav_dist_10pct = pd.concat([avg_trav_dist_10pct, df_avg_trav_dist], axis = 0)

/tmp/ipykernel_7804/4160407999.py:5: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(path, compression= "gzip", sep=";")


In [50]:
avg_trav_time_10pct_sCf = pd.DataFrame()
avg_trav_dist_10pct_sCf = pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" + str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_3765/lausitz-10.0-pct-" + str(elem) +"-fCf_0.1_sCF_0.17783_gS_4711_3765.output_trips.csv.gz"
    temp = pd.read_csv(path, compression= "gzip", sep=";")
    temp["trav_time"] = pd.to_timedelta(temp.trav_time)
    avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
    df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "10-pct", "sample_nr": elem, "alpha" : 0.75, "stuck_time": 30.0, "global_seed": "default"}, index=[elem])
    avg_trav_time_10pct_sCf = pd.concat([avg_trav_time_10pct_sCf, df_avg_trav_time], axis = 0)

    avg_trav_dist = temp["traveled_distance"].mean()
    df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "10-pct", "sample_nr": elem, "alpha" : 0.75, "stuck_time": 30.0, "global_seed": "default"}, index=[elem])
    avg_trav_dist_10pct_sCf = pd.concat([avg_trav_dist_10pct_sCf, df_avg_trav_dist], axis = 0)

/tmp/ipykernel_7804/4189369851.py:5: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(path, compression= "gzip", sep=";")


In [51]:
avg_trav_time_10pct_sT = pd.DataFrame()
avg_trav_dist_10pct_sT = pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-" +str(elem) + "-fCf_sCF_0.1_gS_4711_sT_300.0_3765/lausitz-10-pct-" +str(elem) + "-fCf_sCF_0.1_gS_4711_sT_300.0_3765.output_trips.csv.gz"
    temp = pd.read_csv(path, compression= "gzip", sep=";")
    temp["trav_time"] = pd.to_timedelta(temp.trav_time)
    avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
    df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "10-pct", "sample_nr": elem, "alpha" : 1.0, "stuck_time": 300.0, "global_seed": "default"}, index=[elem])
    avg_trav_time_10pct_sT = pd.concat([avg_trav_time_10pct_sT, df_avg_trav_time], axis = 0)

    avg_trav_dist = temp["traveled_distance"].mean()
    df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "10-pct", "sample_nr": elem, "alpha" : 1.0, "stuck_time": 300.0, "global_seed": "default"}, index=[elem])
    avg_trav_dist_10pct_sT = pd.concat([avg_trav_dist_10pct_sT, df_avg_trav_dist], axis = 0)

/tmp/ipykernel_7804/2542072797.py:5: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(path, compression= "gzip", sep=";")


In [52]:
avg_trav_time_10pct_sT_sCf = pd.DataFrame()
avg_trav_dist_10pct_sT_sCf = pd.DataFrame()
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-" +str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_sT_300.0_3765/lausitz-10-pct-" + str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_sT_300.0_3765.output_trips.csv.gz"
    temp = pd.read_csv(path, compression= "gzip", sep=";")
    temp["trav_time"] = pd.to_timedelta(temp.trav_time)
    avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
    df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "10-pct", "sample_nr": elem, "alpha" : 0.75, "stuck_time": 300.0, "global_seed": "default"}, index=[elem])
    avg_trav_time_10pct_sT_sCf = pd.concat([avg_trav_time_10pct_sT_sCf, df_avg_trav_time], axis = 0)

    avg_trav_dist = temp["traveled_distance"].mean()
    df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "10-pct", "sample_nr": elem, "alpha" : 0.75, "stuck_time": 300.0, "global_seed": "default"}, index=[elem])
    avg_trav_dist_10pct_sT_sCf = pd.concat([avg_trav_dist_10pct_sT_sCf, df_avg_trav_dist], axis = 0)

/tmp/ipykernel_7804/3895922086.py:5: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(path, compression= "gzip", sep=";")


In [53]:
avg_trav_time_10pct_all = pd.concat([avg_trav_time_10pct, avg_trav_time_10pct_sCf, avg_trav_time_10pct_sT, avg_trav_time_10pct_sT_sCf], axis = 0)
avg_trav_dist_10pct_all = pd.concat([avg_trav_dist_10pct, avg_trav_dist_10pct_sCf, avg_trav_dist_10pct_sT, avg_trav_dist_10pct_sT_sCf], axis = 0)

avg_trav_time_10pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_time_all_10pct_samples.csv', index = False) 
avg_trav_dist_10pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_dist_all_10pct_samples.csv', index = False) 

#### 25 pct

In [54]:
# 25 pct
avg_trav_time_25pct = pd.DataFrame()
avg_trav_dist_25pct = pd.DataFrame()
 # 25 pct, alpha = 1
 # import data
path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765/lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765.output_trips.csv.gz"
temp = pd.read_csv(path, compression= "gzip", sep=";")
# travel time
temp["trav_time"] = pd.to_timedelta(temp.trav_time)
avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "25-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": "default"}, index=[0])
avg_trav_time_25pct = pd.concat([avg_trav_time_25pct, df_avg_trav_time ])
# traveled distance
avg_trav_dist = temp["traveled_distance"].mean()
df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "25-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": "default"}, index=[0])
avg_trav_dist_25pct = pd.concat([avg_trav_dist_25pct, df_avg_trav_dist ])

# 25 pct, alpha = 0.75
# import data
path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765/lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765.output_trips.csv.gz"
temp = pd.read_csv(path, compression= "gzip", sep=";")

# travel time
temp["trav_time"] = pd.to_timedelta(temp.trav_time)
avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "25-pct", "sample_nr": 1, "alpha" : 0.75, "stuck_time": 30.0, "global_seed": "default"}, index=[1])
avg_trav_time_25pct = pd.concat([avg_trav_time_25pct, df_avg_trav_time ])

#traveled distance
avg_trav_dist = temp["traveled_distance"].mean()
df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "25-pct", "sample_nr": 1, "alpha" : 0.75, "stuck_time": 30.0, "global_seed": "default"}, index=[1])
avg_trav_dist_25pct = pd.concat([avg_trav_dist_25pct, df_avg_trav_dist ])

# 25 pct, alpha = 1, sT scaled
# import data
path = "/home/lola/math_cluster/output/output-lausitz-25-pct-1-fCf_sCF_0.25_gS_4711_sT_120.0_3765/lausitz-25-pct-1-fCf_sCF_0.25_gS_4711_sT_120.0_3765.output_trips.csv.gz"
temp = pd.read_csv(path, compression= "gzip", sep=";")

# travel time
temp["trav_time"] = pd.to_timedelta(temp.trav_time)
avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "25-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 120.0, "global_seed": "default"}, index=[2])
avg_trav_time_25pct = pd.concat([avg_trav_time_25pct, df_avg_trav_time ])

# traveled distance
avg_trav_dist = temp["traveled_distance"].mean()
df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "25-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 120.0, "global_seed": "default"}, index=[2])
avg_trav_dist_25pct = pd.concat([avg_trav_dist_25pct, df_avg_trav_dist ])

 # 25 pct, alpha = 0.75, sT scaled
# import data
path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765/lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765.output_trips.csv.gz"
temp = pd.read_csv(path, compression= "gzip", sep=";")

# travel time
temp["trav_time"] = pd.to_timedelta(temp.trav_time)
avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "25-pct", "sample_nr": 1, "alpha" : 0.75, "stuck_time": 120.0, "global_seed": "default"}, index=[3])
avg_trav_time_25pct = pd.concat([avg_trav_time_25pct, df_avg_trav_time ])

#traveled distance
avg_trav_dist = temp["traveled_distance"].mean()
df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "25-pct", "sample_nr": 1, "alpha" : 0.75, "stuck_time": 120.0, "global_seed": "default"}, index=[3])
avg_trav_dist_25pct = pd.concat([avg_trav_dist_25pct, df_avg_trav_dist ])


In [55]:
avg_trav_time_25pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_time_all_25pct_samples.csv', index = False) 
avg_trav_dist_25pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_dist_all_25pct_samples.csv', index = False) 

#### 50 pct

In [56]:
# 50 pct
avg_trav_time_50pct = pd.DataFrame()
avg_trav_dist_50pct = pd.DataFrame()
 # 50 pct, alpha = 1
path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765/lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765.output_trips.csv.gz"
temp = pd.read_csv(path, compression= "gzip", sep=";")
temp["trav_time"] = pd.to_timedelta(temp.trav_time)
avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "50-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": "default"}, index=[0])
avg_trav_time_50pct = pd.concat([avg_trav_time_50pct, df_avg_trav_time ])

avg_trav_dist = temp["traveled_distance"].mean()
df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "50-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": "default"}, index=[0])
avg_trav_dist_50pct = pd.concat([avg_trav_dist_50pct, df_avg_trav_dist ])

# 50 pct, alpha = 0.75
path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_0.5_sCF_0.5946_gS_4711_3765/lausitz-50.0-pct-fCf_0.5_sCF_0.5946_gS_4711_3765.output_trips.csv.gz"
temp = pd.read_csv(path, compression= "gzip", sep=";")
temp["trav_time"] = pd.to_timedelta(temp.trav_time)
avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "50-pct", "sample_nr": 1, "alpha" : 0.75, "stuck_time": 30.0, "global_seed": "default"}, index=[1])
avg_trav_time_50pct = pd.concat([avg_trav_time_50pct, df_avg_trav_time ])

avg_trav_dist = temp["traveled_distance"].mean()
df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "50-pct", "sample_nr": 1, "alpha" : 0.75, "stuck_time": 30.0, "global_seed": "default"}, index=[1])
avg_trav_dist_50pct = pd.concat([avg_trav_dist_50pct, df_avg_trav_dist ])

# 50 pct, alpha = 1, sT scaled
path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_sCF_0.5_gS_4711_sT_60.0_3765/lausitz-50-pct-1-fCf_sCF_0.5_gS_4711_sT_60.0_3765.output_trips.csv.gz"
temp = pd.read_csv(path, compression= "gzip", sep=";")
temp["trav_time"] = pd.to_timedelta(temp.trav_time)
avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "50-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 60.0, "global_seed": "default"}, index=[2])
avg_trav_time_50pct = pd.concat([avg_trav_time_50pct, df_avg_trav_time ])

avg_trav_dist = temp["traveled_distance"].mean()
df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "50-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 60.0, "global_seed": "default"}, index=[2])
avg_trav_dist_50pct = pd.concat([avg_trav_dist_50pct, df_avg_trav_dist ])

 # 50 pct, alpha = 0.75, sT scaled
path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_0.5_sCF_0.5946_gS_4711_sT_60.0_3765/lausitz-50-pct-1-fCf_0.5_sCF_0.5946_gS_4711_sT_60.0_3765.output_trips.csv.gz"
temp = pd.read_csv(path, compression= "gzip", sep=";")
temp["trav_time"] = pd.to_timedelta(temp.trav_time)
avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
df_avg_trav_time = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "50-pct", "sample_nr": 1, "alpha" : 0.75, "stuck_time": 60.0, "global_seed": "default"}, index=[3])
avg_trav_time_50pct = pd.concat([avg_trav_time_50pct, df_avg_trav_time ])

avg_trav_dist = temp["traveled_distance"].mean()
df_avg_trav_dist = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "50-pct", "sample_nr": 1, "alpha" : 0.75, "stuck_time": 60.0, "global_seed": "default"}, index=[3])
avg_trav_dist_50pct = pd.concat([avg_trav_dist_50pct, df_avg_trav_dist ])


In [57]:
avg_trav_time_50pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_time_all_50pct_samples.csv', index = False) 
avg_trav_dist_50pct.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_dist_all_50pct_samples.csv', index = False) 

#### 100 pct

In [58]:
# 100 pct
path = "/home/lola/math_cluster/output/output-lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765/lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765.output_trips.csv.gz"
temp = pd.read_csv(path, compression= "gzip", sep=";")
temp["trav_time"] = pd.to_timedelta(temp.trav_time)
avg_trav_time = temp["trav_time"].sum() / temp["trav_time"].shape[0]
avg_trav_time_100pct = pd.DataFrame({"avg_trav_time": avg_trav_time, "sample_size": "100-pct", "sample_nr": 1, "alpha" : 1.0, "stuck_time": 30.0, "global_seed": "default"}, index=[0])

avg_trav_dist = temp["traveled_distance"].mean()
avg_trav_dist_100pct = pd.DataFrame({"avg_trav_dist": avg_trav_dist, "sample_size": "100-pct", "sample_nr": 1, "alpha" :1.0, "stuck_time": 30.0, "global_seed": "default"}, index=[3])


In [59]:
# concat 
avg_trav_time_all = pd.concat([avg_trav_time_1pct_all, avg_trav_time_5pct_all, avg_trav_time_10pct_all, avg_trav_time_25pct, avg_trav_time_50pct, avg_trav_time_100pct], axis=0)
avg_trav_dist_all = pd.concat([avg_trav_dist_1pct_all, avg_trav_dist_5pct_all, avg_trav_dist_10pct_all, avg_trav_dist_25pct, avg_trav_dist_50pct, avg_trav_dist_100pct], axis=0)


In [60]:
avg_trav_time_all["avg_trav_time"] = avg_trav_time_all.avg_trav_time.astype('timedelta64[s]')/  pd.Timedelta(minutes=60)

In [61]:
avg_trav_time_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_time_1_to_100_pct_samples.csv', index = False)
avg_trav_dist_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/avg_trav_dist_1_to_100_pct_sample.csv', index = False)

### 8. Travel Time distribution

#### Travel time categories

In [2]:
def sixMinuteCategoriesUpToTwoHours(df):
    categories = []
    for t in df["trav_time2"]:


        if (0.0 <= t and t < 0.10):
            categories.append("0:00_0:06")
        elif (0.10 <= t and t < 0.20):
            categories.append("0:06_0:12") 
        elif (0.20 <= t and t < 0.30):
            categories.append("0:12_0:18")
        elif (0.30 <= t and t < 0.40):
            categories.append("0:18_0:24")
        elif (0.40 <= t and t < 0.50):
            categories.append("0:24_0:30")
        elif (0.50 <= t and t < 0.60):
            categories.append("0:30_0:36")  
        elif (0.60 <= t and t < 0.70):
            categories.append("0:36_0:42")
        elif (0.70 <= t and t < 0.80):
            categories.append("0:42_0:48")
        elif (0.80 <= t and t < 0.90):
            categories.append("0:48_0:54")
        elif (0.90 <= t and t < 1.00):
            categories.append("0:54_1:00")


        elif (1.0 <= t and t < 1.10):
            categories.append("1:00_1:06")
        elif (1.10 <= t and t < 1.20):
            categories.append("1:06_1:12") 
        elif (1.20 <= t and t < 1.30):
            categories.append("1:12_1:18")
        elif (1.30 <= t and t < 1.40):
            categories.append("1:18_1:24")
        elif (1.40 <= t and t < 1.50):
            categories.append("1:24_1:30")
        elif (1.50 <= t and t < 1.60):
            categories.append("1:30_1:36")  
        elif (1.60 <= t and t < 1.70):
            categories.append("1:36_1:42")
        elif (1.70 <= t and t < 1.80):
            categories.append("1:42_1:48")
        elif (1.80 <= t and t < 1.90):
            categories.append("1:48_1:54")
        elif (1.90 <= t and t < 2.00):
            categories.append("1:54_2:00")

        else: categories.append(">2h")

    temp = pd.DataFrame({ "categories": categories})
    temp = temp.sort_values(by=['categories'])

    freq_6_min_cat = []
    for cat in temp.categories.unique():
        f = temp[(temp["categories"]==cat)].shape[0]
        freq_6_min_cat.append(f)
    
    trav_times_6min_cat = pd.DataFrame({"category":temp.categories.unique(), "freq": freq_6_min_cat })
    return trav_times_6min_cat 


    

#### 1pct

In [3]:
# 1 pct, alpha = 1
trav_time_1pct_categories = []
for elem in range(1,11,1):
    #       /home/lola/math_cluster/output/output-lausitz-1pct-1-fCf_sCF_0.01_gS_default_3765/lausitz-1pct-1-fCf_sCf_0.01_gS_default_3765.output_trips.csv.gz
    path = "/home/lola/math_cluster/output/output-lausitz-1pct-"+ str(elem) +"-fCf_sCF_0.01_gS_default_3765/lausitz-1pct-"+ str(elem) +"-fCf_sCf_0.01_gS_default_3765.output_trips.csv.gz"
    temp = pd.read_csv(path, compression= "gzip", sep=";")
    temp["trav_time"] = pd.to_timedelta(temp.trav_time)
    temp["trav_time2"] = temp.trav_time.astype('timedelta64[s]')/  pd.Timedelta(minutes=60)
    trav_time_cat = sixMinuteCategoriesUpToTwoHours(temp)
    trav_time_cat.rename(columns={"freq": "freq_" + str(elem)}, inplace = True )
    trav_time_1pct_categories.append(trav_time_cat)

    
   

In [4]:
# left join and calculate mean
trav_time_1pct_cat_all = pd.merge(trav_time_1pct_categories[0], trav_time_1pct_categories[1], on = ['category'], how='left')
for elem in range(2,10,1):
    trav_time_1pct_cat_all = pd.merge(trav_time_1pct_cat_all, trav_time_1pct_categories[elem], on = ['category'], how='left')

trav_time_1pct_cat_all['mean'] = trav_time_1pct_cat_all.iloc[:,1:11].mean(axis = 1)
trav_time_1pct_cat_all.insert(12, 'sample_size', "1-pct")
trav_time_1pct_cat_all.insert(13, 'alpha', 1.0)
trav_time_1pct_cat_all.insert(14, 'stuck_time', 30.0)
trav_time_1pct_cat_all.insert(15,'global_seed', "default") 


#### 5 pct

In [5]:
trav_time_5pct_categories = []
for elem in range(1,11,1):
    if (elem == 6):
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765/lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765-2.output_trips.csv.gz"
        temp = pd.read_csv(path, compression = "gzip", sep = ";")
        temp["trav_time"] = pd.to_timedelta(temp.trav_time)
        temp["trav_time2"] = temp.trav_time.astype('timedelta64[s]')/  pd.Timedelta(minutes=60)
        trav_time_cat = sixMinuteCategoriesUpToTwoHours(temp)
        trav_time_cat.rename(columns={"freq": "freq_" + str(elem)}, inplace = True )
        trav_time_5pct_categories.append(trav_time_cat)
    else: 
        # ERROR WRONG PATH, Now corrected
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-" + str(elem) + "-fCf_sCF_0.05_gS_4711_3765/lausitz-5.0-pct-" + str(elem) + "-fCf_sCF_0.05_gS_4711_3765.output_trips.csv.gz"
        temp = pd.read_csv(path, compression= "gzip", sep=";")
        temp["trav_time"] = pd.to_timedelta(temp.trav_time)
        temp["trav_time2"] = temp.trav_time.astype('timedelta64[s]')/  pd.Timedelta(minutes=60)
        trav_time_cat = sixMinuteCategoriesUpToTwoHours(temp)
        trav_time_cat.rename(columns={"freq": "freq_" + str(elem)}, inplace = True )
        trav_time_5pct_categories.append(trav_time_cat)
# left join and calculate mean
trav_time_5pct_cat_all = pd.merge(trav_time_5pct_categories[0], trav_time_5pct_categories[1], on = ['category'], how='left')
for elem in range(2,10,1):
    trav_time_5pct_cat_all = pd.merge(trav_time_5pct_cat_all, trav_time_5pct_categories[elem], on = ['category'], how='left')

trav_time_5pct_cat_all['mean'] = trav_time_5pct_cat_all.iloc[:,1:11].mean(axis = 1)
trav_time_5pct_cat_all.insert(12, 'sample_size', "5-pct")
trav_time_5pct_cat_all.insert(13, 'alpha', 1.0)
trav_time_5pct_cat_all.insert(14, 'stuck_time', 30.0)
trav_time_5pct_cat_all.insert(15,'global_seed', "default") 

#### 10pct

In [6]:
trav_time_10pct_categories = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-" +str(elem) + "-fCf_sCF_0.1_gS_4711_3765/lausitz-10.0-pct-" +str(elem) + "-fCf_sCF_0.1_gS_4711_3765.output_trips.csv.gz"
    temp = pd.read_csv(path, compression= "gzip", sep=";")
    temp["trav_time"] = pd.to_timedelta(temp.trav_time)
    temp["trav_time2"] = temp.trav_time.astype('timedelta64[s]')/  pd.Timedelta(minutes=60)
    trav_time_cat = sixMinuteCategoriesUpToTwoHours(temp)
    trav_time_cat.rename(columns={"freq": "freq_" + str(elem)}, inplace = True )
    trav_time_10pct_categories.append(trav_time_cat)
# left join and calculate mean
trav_time_10pct_cat_all = pd.merge(trav_time_10pct_categories[0], trav_time_10pct_categories[1], on = ['category'], how='left')
for elem in range(2,10,1):
    trav_time_10pct_cat_all = pd.merge(trav_time_10pct_cat_all, trav_time_10pct_categories[elem], on = ['category'], how='left')

trav_time_10pct_cat_all['mean'] = trav_time_10pct_cat_all.iloc[:,1:11].mean(axis = 1)
trav_time_10pct_cat_all.insert(12, 'sample_size', "10-pct")
trav_time_10pct_cat_all.insert(13, 'alpha', 1.0)
trav_time_10pct_cat_all.insert(14, 'stuck_time', 30.0)
trav_time_10pct_cat_all.insert(15,'global_seed', "default") 

/tmp/ipykernel_7233/1743431341.py:4: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(path, compression= "gzip", sep=";")


#### 25 pct

In [7]:
path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765/lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765.output_trips.csv.gz"
t_25pct = pd.read_csv(path, compression= "gzip", sep=";")
t_25pct["trav_time"] = pd.to_timedelta(t_25pct.trav_time)
t_25pct["trav_time2"] = t_25pct.trav_time.astype('timedelta64[s]')/  pd.Timedelta(minutes=60)
trav_time_25pct_categories = sixMinuteCategoriesUpToTwoHours(t_25pct)
trav_time_25pct_categories.rename(columns={"freq": "mean"}, inplace = True )
trav_time_25pct_categories.insert(2, 'sample_size', "25-pct")
trav_time_25pct_categories.insert(3, 'alpha', 1.0)
trav_time_25pct_categories.insert(4, 'stuck_time', 30.0)
trav_time_25pct_categories.insert(5,'global_seed', "default") 

#### 50 pct

In [8]:
path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765/lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765.output_trips.csv.gz"
t_50pct = pd.read_csv(path, compression= "gzip", sep=";")
t_50pct["trav_time"] = pd.to_timedelta(t_50pct.trav_time)
t_50pct["trav_time2"] = t_50pct.trav_time.astype('timedelta64[s]')/  pd.Timedelta(minutes=60)
trav_time_50pct_categories = sixMinuteCategoriesUpToTwoHours(t_50pct)
trav_time_50pct_categories.rename(columns={"freq": "mean"}, inplace = True )
trav_time_50pct_categories.insert(2, 'sample_size', "50-pct")
trav_time_50pct_categories.insert(3, 'alpha', 1.0)
trav_time_50pct_categories.insert(4, 'stuck_time', 30.0)
trav_time_50pct_categories.insert(5,'global_seed', "default") 

#### 100 pct

In [9]:
path = "/home/lola/math_cluster/output/output-lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765/lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765.output_trips.csv.gz"
t_100pct = pd.read_csv(path, compression= "gzip", sep=";")
t_100pct["trav_time"] = pd.to_timedelta(t_100pct.trav_time)
t_100pct["trav_time2"] = t_100pct.trav_time.astype('timedelta64[s]')/  pd.Timedelta(minutes=60)
trav_time_100pct_categories = sixMinuteCategoriesUpToTwoHours(t_100pct)
trav_time_100pct_categories.rename(columns={"freq": "mean"}, inplace = True )
trav_time_100pct_categories.insert(2, 'sample_size', "100-pct")
trav_time_100pct_categories.insert(3, 'alpha', 1.0)
trav_time_100pct_categories.insert(4, 'stuck_time', 30.0)
trav_time_100pct_categories.insert(5,'global_seed', "default") 

#### Concat and write csv

In [10]:
trav_time_categories_all = pd.concat([trav_time_1pct_cat_all, trav_time_5pct_cat_all, trav_time_10pct_cat_all,  trav_time_25pct_categories, trav_time_50pct_categories, trav_time_100pct_categories], axis = 0)
trav_time_categories_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/trav_time_categories_1_100.csv', index = False) 

## 7. Network Congestion Index

#### 1 pct

In [45]:
nci_1pct = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1pct-"+ str(elem) + "-fCf_sCF_0.01_gS_default_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv" 
    temp = pd.read_csv(path, sep=",")
    temp.rename(columns={"congestion_index": "congestion_index_" + str(elem)}, inplace = True)
    nci_1pct.append(temp)

nci_1pct_df = pd.merge(nci_1pct[0], nci_1pct[1], on=['road_type','hour'], how='left')
for elem in range(2,10,1):
    nci_1pct_df = pd.merge(nci_1pct_df, nci_1pct[elem], on = ['road_type', 'hour'], how='left')

nci_1pct_df['congestion_index_mean'] = nci_1pct_df.iloc[: ,2:11].mean(axis=1)
nci_1pct_df.insert(13, 'sample_size', "1-pct")
nci_1pct_df.insert(14, 'alpha', 1.0)
nci_1pct_df.insert(15, 'stuck_time', 30.0)
nci_1pct_df.insert(16,'global_seed', "default")



In [46]:
nci_1pct_sCf = []
for elem in range(1,11,1):
    #       /home/lola/math_cluster/output/output-lausitz-1pct-1-fCf_0.01_sCF_0.03162_gS_default_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv
    path = "/home/lola/math_cluster/output/output-lausitz-1pct-"+ str(elem) + "-fCf_0.01_sCF_0.03162_gS_default_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv" 
    temp = pd.read_csv(path, sep=",")
    temp.rename(columns={"congestion_index": "congestion_index_" + str(elem)}, inplace = True)
    nci_1pct_sCf.append(temp)

nci_1pct_sCf_df = pd.merge(nci_1pct_sCf[0], nci_1pct_sCf[1], on=['road_type','hour'], how='left')
for elem in range(2,10,1):
    nci_1pct_sCf_df = pd.merge(nci_1pct_sCf_df, nci_1pct_sCf[elem], on = ['road_type', 'hour'], how='left')

nci_1pct_sCf_df['congestion_index_mean'] = nci_1pct_sCf_df.iloc[: ,2:11].mean(axis=1)
nci_1pct_sCf_df.insert(13, 'sample_size', "1-pct")
nci_1pct_sCf_df.insert(14, 'alpha', 0.75)
nci_1pct_sCf_df.insert(15, 'stuck_time', 30.0)
nci_1pct_sCf_df.insert(16,'global_seed', "default")


In [47]:
# random global seed
nci_1pct_rGs = []
rGs = [ 4711,3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]
for seed in rGs:
    if (seed == 4711):
        global_seed = "rnd_" + str(seed)
        temp = nci_1pct_df[["road_type", "hour", "congestion_index_1"]].copy()
        nci_1pct_rGs.append(temp)
    elif (seed == 3254):
        global_seed  = "rnd_" + str(seed)
        path ="/home/lola/math_cluster/output/output-lausitz-1pct-1-fCf_sCF_0.01_gS_3254_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
        temp = pd.read_csv(path, sep = ",")
        temp.rename(columns={"congestion_index": "congestion_index_" + str(rGs.index(seed) + 1)}, inplace = True)
        nci_1pct_rGs.append(temp)
    else:
        global_seed  = "rnd_" + str(seed)
        path = "/home/lola/math_cluster/output/output-lausitz-1.0-pct-1-fCf_sCF_0.01_gS_"+str(seed) + "_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
        temp = pd.read_csv(path, sep = ",")
        temp.rename(columns={"congestion_index": "congestion_index_" + str(rGs.index(seed) + 1 )}, inplace = True)
        nci_1pct_rGs.append(temp)

nci_1pct_rGs_df = pd.merge(nci_1pct_rGs[0], nci_1pct_rGs[1], on=['road_type','hour'], how='left')
for elem in range(2,10,1):
    nci_1pct_rGs_df = pd.merge(nci_1pct_rGs_df, nci_1pct_rGs[elem], on = ['road_type', 'hour'], how='left')

nci_1pct_rGs_df['congestion_index_mean'] = nci_1pct_rGs_df.iloc[: ,2:11].mean(axis=1)
nci_1pct_rGs_df.insert(13, 'sample_size', "1-pct")
nci_1pct_rGs_df.insert(14, 'alpha', 1.0)
nci_1pct_rGs_df.insert(15, 'stuck_time', 30.0)
nci_1pct_rGs_df.insert(16,'global_seed', "rnd")



In [48]:
nci_1pct_sT = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-" + str(elem) + "-fCf_sCF_0.01_gS_4711_sT_3000.0_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv" 
    temp = pd.read_csv(path, sep=",")
    temp.rename(columns={"congestion_index": "congestion_index_" + str(elem)}, inplace = True)
    nci_1pct_sT.append(temp)
    
nci_1pct_sT_df = pd.merge(nci_1pct_sT[0], nci_1pct_sT[1], on=['road_type','hour'], how='left')
for elem in range(2,10,1):
    nci_1pct_sT_df = pd.merge(nci_1pct_sT_df, nci_1pct_sT[elem], on = ['road_type', 'hour'], how='left')    
    
nci_1pct_sT_df['congestion_index_mean'] = nci_1pct_sT_df.iloc[: ,2:11].mean(axis=1)
nci_1pct_sT_df.insert(13, 'sample_size', "1-pct")
nci_1pct_sT_df.insert(14, 'alpha', 1.0)
nci_1pct_sT_df.insert(15, 'stuck_time', 3000.0)
nci_1pct_sT_df.insert(16,'global_seed', "default")


In [49]:
nci_1pct_sT_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-1-pct-" + str(elem) + "-fCf_0.01_sCF_0.03162_gS_4711_sT_3000.0_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv" 
    temp = pd.read_csv(path, sep=",")
    temp.rename(columns={"congestion_index": "congestion_index_" + str(elem)}, inplace = True)
    nci_1pct_sT_sCf.append(temp)
    
nci_1pct_sT_sCf_df = pd.merge(nci_1pct_sT_sCf[0], nci_1pct_sT_sCf[1], on=['road_type','hour'], how='left')
for elem in range(2,10,1):
    nci_1pct_sT_sCf_df = pd.merge(nci_1pct_sT_sCf_df, nci_1pct_sT_sCf[elem], on = ['road_type', 'hour'], how='left')    
    
nci_1pct_sT_sCf_df['congestion_index_mean'] = nci_1pct_sT_sCf_df.iloc[: ,2:11].mean(axis=1)
nci_1pct_sT_sCf_df.insert(13, 'sample_size', "1-pct")
nci_1pct_sT_sCf_df.insert(14, 'alpha', 0.75)
nci_1pct_sT_sCf_df.insert(15, 'stuck_time', 3000.0)
nci_1pct_sT_sCf_df.insert(16,'global_seed', "default")

In [50]:
nci_1pct_all = pd.concat([nci_1pct_df, nci_1pct_sCf_df, nci_1pct_rGs_df,  nci_1pct_sT_df, nci_1pct_sT_sCf_df ])

In [51]:
nci_1pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/nci_all_1pct_samples.csv', index = False) 

#### 5 pct

In [52]:
nci_5pct = []
for elem in range(1,11,1):
    if (elem ==6): 
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-6-fCf_sCF_0.05_gS_4711_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
        temp = pd.read_csv(path, sep=",")
        temp.rename(columns={"congestion_index": "congestion_index_" + str(elem)}, inplace = True)
        nci_5pct.append(temp)


    else:
    #       /home/lola/math_cluster/output/output-lausitz-5.0-pct-1-fCf_sCF_0.05_gS_4711_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-"+ str(elem) + "-fCf_sCF_0.05_gS_4711_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv" 
        temp = pd.read_csv(path, sep=",")
        temp.rename(columns={"congestion_index": "congestion_index_" + str(elem)}, inplace = True)
        nci_5pct.append(temp)



nci_5pct_df = pd.merge(nci_5pct[0], nci_5pct[1], on=['road_type','hour'], how='left')
for elem in range(2,10,1):
    nci_5pct_df = pd.merge(nci_5pct_df, nci_5pct[elem], on = ['road_type', 'hour'], how='left')    
    
nci_5pct_df['congestion_index_mean'] = nci_5pct_df.iloc[: ,2:11].mean(axis=1)
nci_5pct_df.insert(13, 'sample_size', "5-pct")
nci_5pct_df.insert(14, 'alpha', 1.0)
nci_5pct_df.insert(15, 'stuck_time', 30.0)
nci_5pct_df.insert(16,'global_seed', "default")


In [53]:
nci_5pct_sCf = []
for elem in range(1,11,1):
    if (elem == 6):
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-6-fCf_0.05_sCF_0.10574_gS_4711_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv" 
        temp = pd.read_csv(path, sep=",")
        temp.rename(columns={"congestion_index": "congestion_index_" + str(elem)}, inplace = True)
        nci_5pct_sCf.append(temp)

    else: 
    #           /home/lola/math_cluster/output/output-lausitz-5.0-pct-7-fCf_0.05_sCF_0.10574_gS_4711_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-"+ str(elem) + "-fCf_0.05_sCF_0.10574_gS_4711_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv" 
        temp = pd.read_csv(path, sep=",")
        temp.rename(columns={"congestion_index": "congestion_index_" + str(elem)}, inplace = True)
        nci_5pct_sCf.append(temp)

nci_5pct_sCf_df = pd.merge(nci_5pct_sCf[0], nci_5pct_sCf[1], on=['road_type','hour'], how='left')
for elem in range(2,10,1):
    nci_5pct_sCf_df = pd.merge(nci_5pct_sCf_df, nci_5pct_sCf[elem], on = ['road_type', 'hour'], how='left')    
    
nci_5pct_sCf_df['congestion_index_mean'] = nci_5pct_sCf_df.iloc[: ,2:11].mean(axis=1)
nci_5pct_sCf_df.insert(13, 'sample_size', "5-pct")
nci_5pct_sCf_df.insert(14, 'alpha',0.75)
nci_5pct_sCf_df.insert(15, 'stuck_time', 30.0)
nci_5pct_sCf_df.insert(16,'global_seed', "default")

In [54]:
# random global seed
nci_5pct_rGs = []

rGs = [ 4711,3254, 2306, 6384,4338, 6003, 5502, 9377, 5621, 9002 ]
for seed in rGs:
    if (seed == 4711):
        global_seed = "rnd_" + str(seed)
        temp = nci_5pct_df[["road_type", "hour", "congestion_index_1"]].copy()
        nci_5pct_rGs.append(temp)
    elif (seed == 3254):
        global_seed  = "rnd_" + str(seed)
        path ="/home/lola/math_cluster/output/output-lausitz-5.0-pct-1-fCf_sCF_0.05_gS_3254_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
        temp = pd.read_csv(path, sep = ",")
        temp.rename(columns={"congestion_index": "congestion_index_" + str(rGs.index(seed) + 1)}, inplace = True)
        nci_5pct_rGs.append(temp)
        
    else:
        if (seed == 6384 or seed == 6003):
            continue
        global_seed  = "rnd_" + str(seed)
        path = "/home/lola/math_cluster/output/output-lausitz-5.0-pct-1-fCf_sCF_0.05_gS_" + str(seed) + "_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
        temp = pd.read_csv(path, sep = ",")
        temp.rename(columns={"congestion_index": "congestion_index_" + str(rGs.index(seed) + 1)}, inplace = True)
        nci_5pct_rGs.append(temp)


nci_5pct_rGs_df = pd.merge(nci_5pct_rGs[0], nci_5pct_rGs[1], on=['road_type','hour'], how='left')
for elem in range(2,8,1):
    nci_5pct_rGs_df = pd.merge(nci_5pct_rGs_df, nci_5pct_rGs[elem], on = ['road_type', 'hour'], how='left')

nci_5pct_rGs_df['congestion_index_mean'] = nci_5pct_rGs_df.iloc[: ,2:9].mean(axis=1)
nci_5pct_rGs_df.insert(11, 'sample_size', "5-pct")
nci_5pct_rGs_df.insert(12, 'alpha', 1.0)
nci_5pct_rGs_df.insert(13, 'stuck_time', 30.0)
nci_5pct_rGs_df.insert(14,'global_seed', "rnd")


#  [Errno 2] No such file or directory: '/home/lola/math_cluster/output/output-lausitz-5.0-pct-1-fCf_sCF_0.05_gS_6384_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv'
# 6003 auch nicht

In [55]:
# 5 pct, alpha 1.0, sT scaled
nci_5pct_sT = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-" + str(elem) + "-fCf_sCF_0.05_gS_4711_sT_600.0_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
    temp = pd.read_csv(path)
    temp.rename(columns={"congestion_index": "congestion_index_" + str(elem)}, inplace = True)
    nci_5pct_sT.append(temp)

nci_5pct_sT_df = pd.merge(nci_5pct_sT[0], nci_5pct_sT[1], on=['road_type','hour'], how='left')
for elem in range(2,10,1):
    nci_5pct_sT_df = pd.merge(nci_5pct_sT_df, nci_5pct_sT[elem], on = ['road_type', 'hour'], how='left')


nci_5pct_sT_df['congestion_index_mean'] = nci_5pct_sT_df.iloc[: ,2:11].mean(axis=1)
nci_5pct_sT_df.insert(13, 'sample_size', "5-pct")
nci_5pct_sT_df.insert(14, 'alpha',1.0)
nci_5pct_sT_df.insert(15, 'stuck_time', 600.0)
nci_5pct_sT_df.insert(16,'global_seed', "default")



In [56]:
nci_5pct_sT_sCf = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-5-pct-" + str(elem) + "-fCf_0.05_sCF_0.10574_gS_4711_sT_600.0_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
    temp = pd.read_csv(path)
    temp.rename(columns={"congestion_index": "congestion_index_" + str(elem)}, inplace = True)
    nci_5pct_sT_sCf.append(temp)

nci_5pct_sT_sCf_df = pd.merge(nci_5pct_sT_sCf[0], nci_5pct_sT_sCf[1], on=['road_type','hour'], how='left')
for elem in range(2,10,1):
    nci_5pct_sT_sCf_df = pd.merge(nci_5pct_sT_sCf_df, nci_5pct_sT_sCf[elem], on = ['road_type', 'hour'], how='left')
    
nci_5pct_sT_sCf_df['congestion_index_mean'] = nci_5pct_sT_sCf_df.iloc[: ,2:11].mean(axis=1)
nci_5pct_sT_sCf_df.insert(13, 'sample_size', "5-pct")
nci_5pct_sT_sCf_df.insert(14, 'alpha',0.75)
nci_5pct_sT_sCf_df.insert(15, 'stuck_time', 600.0)
nci_5pct_sT_sCf_df.insert(16,'global_seed', "default")


In [57]:
nci_5pct_all = pd.concat([nci_5pct_df, nci_5pct_sCf_df,nci_5pct_rGs_df, nci_5pct_sT_df, nci_5pct_sT_sCf_df], axis = 0)

In [58]:
nci_5pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/nci_all_5pct_samples.csv', index = False) 

#### 10 pct

In [59]:
nci_10pct = []
for elem in range(1,11,1):
        path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-"+ str(elem) + "-fCf_sCF_0.1_gS_4711_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv" 
        temp = pd.read_csv(path)
        temp.rename(columns={"congestion_index": "congestion_index_" + str(elem)}, inplace = True)
        nci_10pct.append(temp)

nci_10pct_df = pd.merge(nci_10pct[0], nci_10pct[1], on=['road_type','hour'], how='left')
for elem in range(2,10,1):
    nci_10pct_df = pd.merge(nci_10pct_df, nci_10pct[elem], on = ['road_type', 'hour'], how='left')
    
nci_10pct_df['congestion_index_mean'] = nci_10pct_df.iloc[: ,2:11].mean(axis=1)        
nci_10pct_df.insert(13, 'sample_size', "10-pct")
nci_10pct_df.insert(14, 'alpha',1.0)
nci_10pct_df.insert(15, 'stuck_time', 30.0)
nci_10pct_df.insert(16,'global_seed', "default")



In [60]:
nci_10pct_sCf = []
for elem in range(1,11,1):
        path = "/home/lola/math_cluster/output/output-lausitz-10.0-pct-"+ str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv" 
        temp = pd.read_csv(path)
        temp.rename(columns={"congestion_index": "congestion_index_" + str(elem)}, inplace = True)
        nci_10pct_sCf.append(temp)

nci_10pct_sCf_df = pd.merge(nci_10pct_sCf[0], nci_10pct_sCf[1], on=['road_type','hour'], how='left')
for elem in range(2,10,1):
    nci_10pct_sCf_df = pd.merge(nci_10pct_sCf_df, nci_10pct_sCf[elem], on = ['road_type', 'hour'], how='left')


nci_10pct_sCf_df['congestion_index_mean'] = nci_10pct_sCf_df.iloc[: ,2:11].mean(axis=1)  
nci_10pct_sCf_df.insert(13, 'sample_size', "10-pct")
nci_10pct_sCf_df.insert(14, 'alpha',0.75)
nci_10pct_sCf_df.insert(15, 'stuck_time', 30.0)
nci_10pct_sCf_df.insert(16,'global_seed', "default")


In [61]:
nci_10pct_sT = []
for elem in range(1,11,1):
    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-"+ str(elem) + "-fCf_sCF_0.1_gS_4711_sT_300.0_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
    temp = pd.read_csv(path, sep=",")
    temp.rename(columns={"congestion_index": "congestion_index_" + str(elem)}, inplace = True)
    nci_10pct_sT.append(temp)

nci_10pct_sT_df = pd.merge(nci_10pct_sT[0], nci_10pct_sT[1], on=['road_type','hour'], how='left')
for elem in range(2,10,1):
    nci_10pct_sT_df = pd.merge(nci_10pct_sT_df, nci_10pct_sT[elem], on = ['road_type', 'hour'], how='left')


nci_10pct_sT_df['congestion_index_mean'] = nci_10pct_sT_df.iloc[: ,2:11].mean(axis=1)  


nci_10pct_sT_df.insert(13, 'sample_size', "10-pct")
nci_10pct_sT_df.insert(14, 'alpha',1.0)
nci_10pct_sT_df.insert(15, 'stuck_time', 300.0)
nci_10pct_sT_df.insert(16,'global_seed', "default")


In [62]:
nci_10pct_sT_sCf = []
for elem in range(1,11,1):
    if (elem == 2):
        continue
    path = "/home/lola/math_cluster/output/output-lausitz-10-pct-" +str(elem) + "-fCf_0.1_sCF_0.17783_gS_4711_sT_300.0_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
    temp = pd.read_csv(path, sep=",")
    temp.rename(columns={"congestion_index": "congestion_index_" + str(elem)}, inplace = True)
    nci_10pct_sT_sCf.append(temp)


nci_10pct_sT_sCf_df = pd.merge(nci_10pct_sT_sCf[0], nci_10pct_sT_sCf[1], on=['road_type','hour'], how='left')
for elem in range(2,9,1):
    nci_10pct_sT_sCf_df = pd.merge(nci_10pct_sT_sCf_df, nci_10pct_sT_sCf[elem], on = ['road_type', 'hour'], how='left')


nci_10pct_sT_sCf_df['congestion_index_mean'] = nci_10pct_sT_sCf_df.iloc[: ,2:10].mean(axis=1) 
nci_10pct_sT_sCf_df.insert(12, 'sample_size', "10-pct")
nci_10pct_sT_sCf_df.insert(13, 'alpha',0.75)
nci_10pct_sT_sCf_df.insert(14, 'stuck_time', 300.0)
nci_10pct_sT_sCf_df.insert(15,'global_seed', "default")


# 2 nicht da, fehlerhaft!!

In [63]:
nci_10pct_all = pd.concat([nci_10pct_df, nci_10pct_sCf_df, nci_10pct_sT_df, nci_10pct_sT_sCf_df], axis = 0)

In [64]:
nci_10pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/nci_10pct_samples_some_missing.csv', index = False) 

#### 25 pct

In [65]:
path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_sCF_0.25_gS_4711_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
nci_25pct = pd.DataFrame(pd.read_csv(path, sep = ","))
nci_25pct.rename(columns={"congestion_index": "congestion_index_mean"}, inplace = True)
nci_25pct.insert(3, 'sample_size', "25-pct")
nci_25pct.insert(4, 'alpha',1.0)
nci_25pct.insert(5, 'stuck_time', 30.0)
nci_25pct.insert(6,'global_seed', "default")


In [66]:


path = "/home/lola/math_cluster/output/output-lausitz-25.0-pct-fCf_0.25_sCF_0.35355_gS_4711_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
nci_25pct_sCf = pd.DataFrame(pd.read_csv(path, sep = ","))
nci_25pct_sCf.rename(columns={"congestion_index": "congestion_index_mean"}, inplace = True)
nci_25pct_sCf.insert(3, 'sample_size', "25-pct")
nci_25pct_sCf.insert(4, 'alpha',0.75)
nci_25pct_sCf.insert(5, 'stuck_time', 30.0)
nci_25pct_sCf.insert(6,'global_seed', "default")

path = "/home/lola/math_cluster/output/output-lausitz-25-pct-1-fCf_sCF_0.25_gS_4711_sT_120.0_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
nci_25pct_sT = pd.DataFrame(pd.read_csv(path, sep = ","))
nci_25pct_sT.rename(columns={"congestion_index": "congestion_index_mean"}, inplace = True)
nci_25pct_sT.insert(3, 'sample_size', "25-pct")
nci_25pct_sT.insert(4, 'alpha',1.0)
nci_25pct_sT.insert(5, 'stuck_time',120.0)
nci_25pct_sT.insert(6,'global_seed', "default")

#path = ""
#nci_25pct_sT_sCf = pd.DataFrame(pd.read_csv(path, sep = ","))
#nci_25pct_sT_sCf.insert(3, 'sample_size', "25-pct")
#nci_25pct_sT_sCf.insert(4, 'sample_nr', 1)
#nci_25pct_sT_sCf.insert(5, 'alpha',0.75)
#nci_25pct_sT_sCf.insert(6, 'stuck_time',120.0)
#nci_25pct_sT_sCf.insert(7,'global_seed', "default")

# FEHLT


In [67]:
nci_25pct_all = pd.concat([nci_25pct, nci_25pct_sCf, nci_25pct_sT], axis = 0)

In [68]:
nci_25pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/nci_25pct_samples_sCf_sT_missing.csv', index = False) 

#### 50 pct

In [69]:
path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_sCF_0.5_gS_4711_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
nci_50pct = pd.DataFrame(pd.read_csv(path, sep = ","))
nci_50pct.rename(columns={"congestion_index": "congestion_index_mean"}, inplace = True)
nci_50pct.insert(3, 'sample_size', "50-pct")
nci_50pct.insert(4, 'alpha',1.0)
nci_50pct.insert(5, 'stuck_time', 30.0)
nci_50pct.insert(6,'global_seed', "default")


path = "/home/lola/math_cluster/output/output-lausitz-50.0-pct-fCf_0.5_sCF_0.5946_gS_4711_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
nci_50pct_sCf = pd.DataFrame(pd.read_csv(path, sep = ","))
nci_25pct_sCf.rename(columns={"congestion_index": "congestion_index_mean"}, inplace = True)
nci_50pct_sCf.insert(3, 'sample_size', "50-pct")
nci_50pct_sCf.insert(4, 'alpha',0.75)
nci_50pct_sCf.insert(5, 'stuck_time', 30.0)
nci_50pct_sCf.insert(6,'global_seed', "default")


path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_sCF_0.5_gS_4711_sT_60.0_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
nci_50pct_sT = pd.DataFrame(pd.read_csv(path, sep = ","))
nci_50pct_sT.rename(columns={"congestion_index": "congestion_index_mean"}, inplace = True)
nci_50pct_sT.insert(3, 'sample_size', "50-pct")
nci_50pct_sT.insert(4, 'alpha',1.0)
nci_50pct_sT.insert(5, 'stuck_time', 60.0)
nci_50pct_sT.insert(6,'global_seed', "default")

path = "/home/lola/math_cluster/output/output-lausitz-50-pct-1-fCf_0.5_sCF_0.5946_gS_4711_sT_60.0_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
nci_50pct_sT_sCf = pd.DataFrame(pd.read_csv(path, sep = ","))
nci_50pct_sT_sCf.rename(columns={"congestion_index": "congestion_index_mean"}, inplace = True)
nci_50pct_sT_sCf.insert(3, 'sample_size', "50-pct")
nci_50pct_sT_sCf.insert(4, 'alpha',0.75)
nci_50pct_sT_sCf.insert(5, 'stuck_time', 60.0)
nci_50pct_sT_sCf.insert(6,'global_seed', "default")



In [70]:
nci_50pct_all = pd.concat([nci_50pct, nci_50pct_sCf, nci_50pct_sT, nci_50pct_sT_sCf], axis = 0)

In [71]:
nci_50pct_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/nci_50pct_samples.csv', index = False) 

#### 100 pct

In [72]:
path = "/home/lola/math_cluster/output/output-lausitz-100.0-pct-fCf_sCF_1.0_gS_4711_3765/analysis/traffic/traffic_stats_by_road_type_and_hour.csv"
nci_100pct = pd.DataFrame(pd.read_csv(path, sep = ","))
nci_100pct.rename(columns={"congestion_index": "congestion_index_mean"}, inplace = True)
nci_100pct.insert(3, 'sample_size', "100-pct")
nci_100pct.insert(4, 'alpha',1.0)
nci_100pct.insert(5, 'stuck_time', 30.0)
nci_100pct.insert(6,'global_seed', "default")

#### concat and write csv

In [73]:
nci_all = pd.concat([nci_1pct_all, nci_5pct_all, nci_10pct_all, nci_25pct_all, nci_50pct_all, nci_100pct], axis = 0)

In [74]:
nci_all

,road_type,hour,congestion_index_1,congestion_index_2,congestion_index_3,congestion_index_4,congestion_index_5,congestion_index_6,congestion_index_7,congestion_index_8,congestion_index_9,congestion_index_10,congestion_index_mean,sample_size,alpha,stuck_time,global_seed,congestion_index
0,all,0,0.801976,0.801968,0.801976,0.801976,0.801976,0.801976,0.801976,0.801976,0.801976,0.801976,0.801975,1-pct,1.0,30.0,default,NaN
1,all,1,0.801976,0.801968,0.801976,0.801972,0.801976,0.801976,0.801976,0.801976,0.801976,0.801975,0.801975,1-pct,1.0,30.0,default,NaN
2,all,2,0.801976,0.801946,0.801976,0.801961,0.801974,0.801975,0.801972,0.801976,0.801974,0.801972,0.801970,1-pct,1.0,30.0,default,NaN
3,all,3,0.801939,0.801905,0.801965,0.801924,0.801942,0.801929,0.801944,0.801882,0.801942,0.801906,0.801930,1-pct,1.0,30.0,default,NaN
4,all,4,0.801758,0.801818,0.801662,0.801832,0.801710,0.801658,0.801599,0.801789,0.801636,0.801557,0.801718,1-pct,1.0,30.0,default,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,unclassified,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.744103,100-pct,1.0,30.0,default,NaN
332,unclassified,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.738027,100-pct,1.0,30.0,default,NaN
333,unclassified,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.742919,100-pct,1.0,30.0,default,NaN
334,unclassified,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.745181,100-pct,1.0,30.0,default,NaN


In [75]:
nci_all.to_csv('/home/lola/Nextcloud/Masterarbeit/03_Outputs/nci_100pct_samples_some_missing.csv', index = False) 